<a href="https://www.kaggle.com/code/oswind/stockchat-agents-edition?scriptVersionId=281526760" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Environment Setup

In [1]:
# Setup the notebook based on running environment.
import os
# Optional: Enable telemetry in browser_use and chromadb.
os.environ["ANONYMIZED_TELEMETRY"] = "false"
# Check for kaggle environment.
if os.getenv("KAGGLE_KERNEL_RUN_TYPE"):
    # Kaggle Run: update the system.
    !pip uninstall -qqy google-ai-generativelanguage pydrive2 tensorflow tensorflow-decision-forests cryptography pyOpenSSL langchain langchain-core nltk ray click google-generativeai google-cloud-translate datasets cesium bigframes plotnine mlxtend fastai spacy thinc google-colab gcsfs jupyter-kernel-gateway
    !pip install -qU posthog\<6.0.0 google-genai==1.50.0 chromadb==0.6.3 opentelemetry-proto==1.37.0
    !pip install -qU langchain-community langchain-text-splitters wikipedia lmnr[all] google-adk google-cloud-translate
    from kaggle_secrets import UserSecretsClient # type: ignore
    from jupyter_server.serverapp import list_running_servers # type: ignore
else:
    # Mock the kaggle secrets client.
    class UserSecretsClient:
        @classmethod
        def set_secret(cls, id: str, value: str):
            os.environ[id] = value
        @classmethod
        def get_secret(cls, id: str):
            try:
                return os.environ[id]
            except KeyError as e:
                print(f"KeyError: authentication token for {id} is undefined")
    # Local Run: update the venv.
    %pip install -qU posthog\<6.0.0 google-genai==1.50.0 chromadb==0.6.3 opentelemetry-proto==1.37.0
    %pip install -qU langchain-community langchain-text-splitters wikipedia pandas google-api-core "lmnr[all]" browser-use ollama google-adk
    from browser_use import Agent as BrowserAgent

import ast, chromadb, json, logging, pandas, platform, pytz, re, requests, threading, time, warnings, wikipedia
from bs4 import Tag
from chromadb import Documents, Embeddings
from datetime import datetime, timedelta
from dateutil.parser import parse
from enum import Enum
from google.adk.apps.app import App
from google.adk.sessions import InMemorySessionService, BaseSessionService as SessionService
from google.adk.runners import Runner, Event
from google import genai
from google.api_core import retry, exceptions
from google.genai.models import Models
from google.genai import types, errors
from IPython.display import Markdown, display, HTML
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters.html import HTMLSemanticPreservingSplitter
from langchain_text_splitters.json import RecursiveJsonSplitter
from lmnr import Laminar
from math import inf
from pydantic import BaseModel, field_validator
from threading import Timer
from tqdm import tqdm
from typing import Optional, Callable, NewType, NamedTuple
from wikipedia.exceptions import DisambiguationError, PageError

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.3/257.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 51.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 82.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108

In [2]:
# Prepare the Gemini api for use.
# Setup a retry helper for generation not run through the below api-helper.
is_retriable = lambda e: (isinstance(e, errors.APIError) and e.code in {429, 503, 500})
Models.generate_content = retry.Retry(predicate=is_retriable)(Models.generate_content)
Models.embed_content = retry.Retry(predicate=is_retriable)(Models.embed_content)

# Activate Laminar auto-instrumentation.
try:
    Laminar.initialize(project_api_key=UserSecretsClient().get_secret("LMNR_PROJECT_API_KEY"))
except:
    print("Skipping Laminar.initialize()")

class GeminiModel:
    def __init__(self, rpm: list, tpm: list, rpd: list):
        self.rpm = rpm # requests per minute
        self.tpm = tpm # tokens per minute in millions
        self.rpd = rpd # requests per day
        self.err = [0,0] # validation, api_related

# A python api-helper with model fail-over/chaining/retry support.
GeminiEmbedFunction = NewType("GeminiEmbedFunction", None) # forward-decl
class Api:
    gen_limit_in = 1048576
    emb_limit_in = 2048
    gen_model = {
        "gemini-2.5-flash": GeminiModel([10,1000,2000,10000],[.25,1,3,8],[250,10000,100000,inf]), # stable: 10 RPM/250K TPM/250 RPD
        "gemini-2.5-flash-preview-09-2025": GeminiModel([10,1000,2000,10000],[.25,1,3,8],[250,10000,100000,inf]), # exp: 10 RPM/250K TPM/250 RPD
        "gemini-2.0-flash-exp": GeminiModel([10,10,10,10],[.25,.25,.25,.25],[200,500,500,500]), # latest w/thinking: 10 RPM/250K TPM/200 RPD
        "gemini-2.0-flash": GeminiModel([15,2000,10000,30000],[1,4,10,30],[200,inf,inf,inf]), # stable wo/thinking: 15 RPM/1M TPM/200 RPD
        "gemini-2.5-flash-lite": GeminiModel([15,4000,10000,30000],[.25,4,10,30],[1000,inf,inf,inf]), # stable: 15 RPM/250K TPM/1K RPD
        "gemini-2.5-flash-lite-preview-09-2025": GeminiModel([15,4000,10000,30000],[.25,4,10,30],[1000,inf,inf,inf]), # exp: 15 RPM/250K TPM/1K RPD
        "gemini-2.5-pro": GeminiModel([5,150,1000,2000],[.125,2,5,8],[100,10000,50000,inf]), # stable: 5 RPM/250K TPM/100 RPD
    }
    gen_local = ["gemma3n:e4b","gemma3:12b-it-qat"]
    default_local = 0
    default_model = []
    embed_model = "gemini-embedding-001", GeminiModel([100,3000,5000,10000],[.03,1,5,10],[1000,inf,inf,inf]) # stable: 100 RPM/30K TPM/1000 RPD/100 per batch
    embed_local = False
    error_total = 0
    min_rpm = 3
    min_tpm = 40000
    dt_between = 2.0
    errored = False
    running = False
    dt_err = 45.0
    dt_rpm = 60.0

    @classmethod
    def get(cls, url: str):
        # Create a header matching the OS' tcp-stack fingerprint.
        system_ua = None
        match platform.system():
            case 'Linux':
                system_ua = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'
            case 'Darwin':
                system_ua = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 15_7_2) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/26.0 Safari/605.1.15'
            case 'Windows':
                system_ua = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'
        try:
            request = requests.get(url, headers={'User-Agent': system_ua})
            if request.status_code != requests.codes.ok:
                print(f"Api.get() returned status {request.status_code}")
            return request.text
        except Exception as e:
            raise e

    class Limit(Enum):
        FREE = 0
        TIER_1 = 1
        TIER_2 = 2
        TIER_3 = 3
    
    class Model(Enum):
        GEN = 1
        EMB = 2
        LOC = 3

    class Const(Enum):
        STOP = "I don't know."
        METRIC_BATCH = 20
        SERIES_BATCH = 40
        EMBED_BATCH = 100
        CHUNK_MAX = 1500

        @classmethod
        def Stop(cls):
            return cls.STOP.value

        @classmethod
        def MetricBatch(cls):
            return cls.METRIC_BATCH.value

        @classmethod
        def SeriesBatch(cls):
            return cls.SERIES_BATCH.value

        @classmethod
        def EmbedBatch(cls):
            return cls.EMBED_BATCH.value

        @classmethod
        def ChunkMax(cls):
            return cls.CHUNK_MAX.value
    
    class Env(NamedTuple): # Make init args immutable.
        CLIENT: genai.Client
        API_LIMIT: int
        GEN_DEFAULT: str

    def __init__(self, with_limit: Limit | int, default_model: str):
        if default_model in self.gen_model.keys():
            self.write_lock = threading.RLock()
            try:
                if isinstance(with_limit, int) and with_limit in [id.value for id in Api.Limit]:
                    limit = with_limit
                else:
                    limit = with_limit.value
            except Exception as e:
                print(f"Api.__init__: {with_limit} is not a valid limit")
            else:
                self.args = Api.Env(
                    genai.Client(api_key=UserSecretsClient().get_secret("GOOGLE_API_KEY")),
                    limit, default_model)
            self.m_id = list(self.gen_model.keys()).index(default_model)
            self.default_model.append(default_model)
            self.update_quota()
            self.s_embed = GeminiEmbedFunction(self.args.CLIENT, semantic_mode = True) # type: ignore
            logging.getLogger("google_genai").setLevel(logging.WARNING) # suppress info on generate
        else:
            print(f"Api.__init__: {default_model} not found in gen_model.keys()")
        

    def __call__(self, model: Model) -> str:
        if model == self.Model.GEN:
            return "models/" + list(self.gen_model.keys())[self.m_id]
        elif model == self.Model.LOC:
            return self.gen_local[self.default_local]
        else:
            return "models/" + self.embed_model[0] if not self.embed_local else "embeddinggemma:latest"

    def push_default_model(self, model_id: str):
        if model_id in self.gen_model.keys():
            self.write_lock.acquire()
            self.stop_running()
            self.default_model.append(model_id)
            self.m_id = list(self.gen_model.keys()).index(model_id)
            self.write_lock.release()
        else:
            print(f"{model_id} not found in gen_model.keys()")

    def pop_default_model(self):
        if len(self.default_model) > 1:
            self.write_lock.acquire()
            self.stop_running()
            self.default_model.pop(-1)
            self.m_id = list(self.gen_model.keys()).index(self.default_model[-1])
            self.write_lock.release()

    def retriable(self, retry_fn: Callable, *args, **kwargs):
        tries = 3*len(self.gen_model.keys())
        for attempt in range(tries):
            try:
                self.write_lock.acquire()
                token_use = self.token_count(kwargs["contents"])
                if self.gen_rpm > self.min_rpm and token_use <= self.token_quota and self.token_quota > self.min_tpm:
                    self.token_quota -= token_use
                    self.gen_rpm -= 1
                else:
                    self.on_error(kwargs)
                if not self.running and not self.errored:
                    self.rpm_timer = Timer(self.dt_rpm, self.refill_rpm)
                    self.rpm_timer.start()
                    self.running = True
                return retry_fn(*args, **kwargs)
            except (errors.APIError, exceptions.RetryError) as api_error:
                if isinstance(api_error, errors.APIError):
                    is_retry = api_error.code in {429, 503, 500, 400} # code 400 when TPM exceeded
                    if api_error.code == 400:
                        print(f"retriable.api_error: token limit exceeded ({token_use})")
                    else:
                        print(f"retriable.api_error({api_error.code}): {str(api_error)}")
                    if not is_retry or attempt == tries:
                        raise api_error
                self.on_error(kwargs)
            except Exception as e:
                print(f"retriable.exception: {str(e)}")
                self.on_error(kwargs)
            finally:
                self.write_lock.release()

    def on_error(self, kwargs):
        self.generation_fail()
        kwargs["model"] = self(Api.Model.GEN)
        time.sleep(self.dt_between)

    def stop_running(self):
        if self.running:
            self.rpm_timer.cancel()
            self.running = False

    def validation_fail(self):
        list(self.gen_model.values())[self.m_id].err[0] += 1
        self.error_total += 1

    def generation_fail(self):
        self.stop_running()
        self.save_error()
        self.next_model()
        print("Api.generation_fail.next_model: model is now", list(self.gen_model.keys())[self.m_id])
        if not self.errored:
            self.error_timer = Timer(self.dt_err, self.zero_error)
            self.error_timer.start()
            self.errored = True

    def save_error(self):
        list(self.gen_model.values())[self.m_id].err[1] += 1
        self.error_total += 1

    def next_model(self):
        self.m_id = (self.m_id+1)%len(self.gen_model.keys())
        self.update_quota()

    def refill_rpm(self):
        self.running = False
        self.update_quota()
        print("Api.refill_rpm", self.gen_rpm)

    def zero_error(self):
        self.errored = False
        self.m_id = list(self.gen_model.keys()).index(self.default_model[-1])
        self.update_quota()
        print("Api.zero_error: model is now", list(self.gen_model.keys())[self.m_id])

    def update_quota(self):
        self.gen_rpm = list(self.gen_model.values())[self.m_id].rpm[self.args.API_LIMIT]
        self.token_quota = list(self.gen_model.values())[self.m_id].tpm[self.args.API_LIMIT]*1_000_000

    def token_count(self, expr: str | list):
        count = self.args.CLIENT.models.count_tokens(
            model=self(Api.Model.GEN),
            contents=json.dumps(expr) if isinstance(expr, str) else str(expr))
        return count.total_tokens

    def errors(self):
        errors = {"total": self.error_total, "by_model": {}}
        for m_code, m in self.gen_model.items():
            errors["by_model"].update({
                m_code: {
                    "api_related": m.err[1],
                    "validation": m.err[0]
                }})
        return errors

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def similarity(self, content: list):
        return self.s_embed.sts(content) # type: ignore

Skipping Laminar.initialize()


In [3]:
# Define the embedding function.
api = NewType("api", Api) # type: ignore (forward-decl)
class GeminiEmbedFunction:
    document_mode = True  # Generate embeddings for documents (T,F), or queries (F,F).
    semantic_mode = False # Semantic text similarity mode is exclusive (F,T).
    
    def __init__(self, genai_client, semantic_mode: bool = False):
        self.client = genai_client
        if semantic_mode:
            self.document_mode = False
            self.semantic_mode = True

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def __embed__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        elif not self.document_mode and not self.semantic_mode:
            embedding_task = "retrieval_query"
        elif not self.document_mode and self.semantic_mode:
            embedding_task = "semantic_similarity"
        partial = self.client.models.embed_content(
            model=api(Api.Model.EMB),
            contents=input,
            config=types.EmbedContentConfig(task_type=embedding_task)) # type: ignore
        return [e.values for e in partial.embeddings]
    
    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def __call__(self, input: Documents) -> Embeddings:
        try:
            response = []
            for i in range(0, len(input), Api.Const.EmbedBatch()):  # Gemini max-batch-size is 100.
                response += self.__embed__(input[i:i + Api.Const.EmbedBatch()])
            return response
        except Exception as e:
            print(f"caught exception of type {type(e)}\n{e}")
            raise e

    def sts(self, content: list) -> float:
        df = pandas.DataFrame(self(content), index=content)
        score = df @ df.T
        return score.iloc[0].iloc[1]

## Set Gemini API Limit

In [4]:
# Instantiate the api-helper with usage limit => FREE.
# Optional: Set limit here to one of [FREE,TIER_1,TIER_2,TIER_3]
api = Api(with_limit=Api.Limit.FREE, default_model="gemini-2.5-flash")
# Export api environment for agent.
os.environ["API_LIMIT"]=str(api.args.API_LIMIT)
os.environ["GEN_DEFAULT"]=api.args.GEN_DEFAULT
# Cleanup old vector_db instances.
!rm -rf vector_db

# Gemini Baseline Check

In [5]:
# This is an accurate retelling of events. 
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
    temperature=0.0
)

chat = api.args.CLIENT.chats.create(
    model=api(Api.Model.GEN),
    config=config_with_search,
    history=[]) # Ignoring the part about dark elves, and tengwar.

response = chat.send_message('Do you know anything about the stock market?')
Markdown(response.text)

The stock market is a global network of exchanges and over-the-counter (OTC) marketplaces where investors buy and sell shares of publicly traded companies. These shares, also known as equities, represent partial ownership in a company.

**How the Stock Market Works**
The stock market operates through a system of supply and demand, which determines the price of each security. When demand from buyers exceeds the supply from sellers, the price typically rises, and vice versa.

The process generally involves two main markets:
*   **Primary Market:** This is where companies initially issue new stocks to the public through a process called an Initial Public Offering (IPO) to raise capital for growth and expansion.
*   **Secondary Market:** After the IPO, these shares are traded among investors on stock exchanges like the New York Stock Exchange (NYSE) or Nasdaq. The company is no longer directly involved in these subsequent transactions.

**Purpose of the Stock Market**
The stock market serves two crucial functions:
1.  **Capital Raising for Companies:** It allows businesses to raise money from the public by selling shares of ownership, which they can then use to fund and expand their operations.
2.  **Wealth Creation for Investors:** It provides individuals and institutions with an opportunity to invest in companies and potentially grow their wealth through capital gains (when stock prices increase) or dividends (a share of the company's profits).

**Key Components and Participants**
*   **Stock Exchanges:** These are organized platforms, often electronic, where stocks and other securities are bought and sold. They provide the infrastructure for trades, maintain orderly markets, and ensure compliance with regulations.
*   **Brokers:** These intermediaries execute buy and sell orders on behalf of investors.
*   **Investors:** Participants range from individual retail investors to large institutional investors like pension funds, mutual funds, insurance companies, and hedge funds.

**Types of Investments**
While "stock market" often refers to stocks, it encompasses various investment types:
*   **Common Stock:** Represents partial ownership and typically grants voting rights on corporate decisions and the potential for dividends.
*   **Preferred Stock:** Also represents ownership but usually doesn't come with voting rights. Preferred shareholders have a higher claim on dividends and assets in case of liquidation compared to common stockholders.
*   **Exchange-Traded Funds (ETFs):** These trade like stocks but allow investors to own a basket of different stocks, commodities, or other assets.
*   **Mutual Funds:** Professionally managed portfolios that pool money from multiple investors to invest in a diversified collection of securities.
*   **Bonds:** While primarily debt instruments, they are often discussed alongside stocks as investment options. Investing in bonds means lending money to a government or corporation for regular interest payments.

**Factors Influencing Stock Prices**
Stock market movements are influenced by a variety of factors, including macroeconomic indicators (like interest rates, inflation, and GDP growth), company-specific news (such as earnings reports and product launches), political events, and geopolitical tensions.

In [6]:
response = chat.send_message('I have an interest in AMZN stock')
Markdown(response.text)

Amazon.com Inc. (NASDAQ: AMZN) is a multinational technology company known for its extensive operations in e-commerce, cloud computing (Amazon Web Services - AWS), digital streaming, and artificial intelligence. Founded by Jeff Bezos in 1994 as an online bookstore, it has grown into one of the world's largest internet retail platforms and a significant player in the tech industry.

As of November 24, 2025, Amazon's stock (AMZN) is trading around $221.44. The stock has fluctuated between $221.60 and $226.79 today, and its 52-week range has been between $161.38 and $258.60. The company's market capitalization stands at approximately $2.36 trillion, making it one of the most valuable companies globally.

Amazon's business is diversified across several key segments:
*   **North America and International:** These segments encompass its vast online retail operations, offering a wide range of products and services.
*   **Amazon Web Services (AWS):** This cloud computing platform provides on-demand computing resources and is a significant driver of Amazon's profitability, often generating the majority of its operating income.
*   **Other ventures:** Amazon also has a strong presence in digital streaming, online advertising, and artificial intelligence. The company recently announced plans to invest up to $50 billion to expand its AI and supercomputing infrastructure for U.S. government clients.

In terms of recent performance, AMZN stock has seen a 1.73% rise over the last month and a 13.50% increase over the past year. The company's most recent stock split was a 20-for-1 split in June 2022, which aimed to make shares more accessible to retail investors.

Analyst sentiment for AMZN stock is generally positive, with a consensus rating of "Strong Buy" from numerous analysts. The average analyst price target for Amazon stock ranges from approximately $280.47 to $297.64, suggesting a potential increase from its current price over the next year. The highest analyst price target is $340.00, while the lowest is $195.00.

In [7]:
response = chat.send_message('''Tell me about AMZN current share price, short-term trends, and bullish versus bearish predictions''')
Markdown(response.text)

Amazon.com Inc. (NASDAQ: AMZN) is currently trading around **$225.01** as of November 24, 2025, reflecting a 2.32% increase in the past 24 hours.

**Short-Term Trends:**
In the short term, AMZN has experienced some fluctuations. While it has seen a 1.73% rise over the last month, the stock has fallen by -3.19% compared to the previous week. The stock's price has been moving between $222.27 and $227.27 today. From a technical perspective, Amazon is in a short-term downtrend on the daily chart, although it remains within a broader long-term uptrend. The price has been making lower highs and lower lows over the last two weeks, with momentum indicators showing some weakening. However, a buy signal was issued from a pivot bottom point on November 20, 2025, and the stock has risen 1.63% since then, with rising volume, which is considered a positive technical signal. The stock is currently below its 5, 20, and 50-day exponential moving averages, indicating a strongly bearish short-term trend, but it is also experiencing buying pressure.

**Bullish Predictions:**
*   **Analyst Consensus:** The overall sentiment from analysts is strongly bullish. Amazon has an average brokerage recommendation of "Strong Buy" from 58 brokerage firms, with 50 strong buy ratings and six buy ratings.
*   **Price Targets:** The average analyst price target for AMZN ranges from approximately $280.47 to $297.64, suggesting a potential upside of 23.82% to 34.06% from its current price over the next year. The highest price target among analysts is $360.00.
*   **Growth Drivers:** Bullish arguments are supported by the continued growth of Amazon Web Services (AWS), which remains a significant profit driver for the company. Amazon's dominance in e-commerce and its expanding advertising business also contribute to a positive outlook. The company's recent announcement to invest up to $50 billion to expand its AI and supercomputing infrastructure for U.S. government clients is also seen as a positive development.
*   **Long-Term Potential:** Historically, AMZN has shown strong long-term performance, rising by an average of 55.1% over a 52-week period based on the past 28 years.

**Bearish Predictions:**
*   **Technical Indicators:** Some technical indicators suggest caution. The stock holds sell signals from both short and long-term Moving Averages, and there is a general sell signal from the relation where the long-term average is above the short-term average. Additionally, there is a sell signal from the 3-month Moving Average Convergence Divergence (MACD).
*   **Valuation Concerns:** The price-to-earnings ratio of 33.10 may suggest that the stock is overvalued compared to its earnings, which could deter value-focused investors.
*   **Competition and Macro Factors:** While AWS is strong, its growth has been slowing, leading to concerns about losing market share to competitors like Microsoft's Azure. Broader macroeconomic conditions, such as high inflation, changes in consumer spending, and interest rate hikes, could also influence Amazon's revenue growth and profitability, particularly in its retail operations.
*   **Recent Pullback:** The stock has pulled back about 13% from its early-November peak of $258.60 to the current trading range, despite strong Q3 2025 results. This pullback reflects a repricing of risk rather than a failure of the business, but it highlights potential volatility.

In [8]:
response = chat.send_message('''What is mgm studio's stock ticker symbol?''')
Markdown(response.text)

The stock ticker symbol "MGM" on the New York Stock Exchange (NYSE) belongs to **MGM Resorts International**.

It's important to note that MGM Resorts International is a hospitality and entertainment company known for its resorts and casinos. While "Amazon MGM Studios" is mentioned in the context of co-produced series, the "MGM" ticker symbol itself is for the resorts company, not directly for the film studio as a separate publicly traded entity.

In [9]:
response = chat.send_message('''What was the last open,close,high,low data for AMZN again?''')
Markdown(response.text)

Here is the last available open, close, high, and low data for AMZN as of November 21, 2025:

*   **Open:** $216.34
*   **Close:** $220.69
*   **High:** $222.21
*   **Low:** $215.18

In [10]:
response = chat.send_message('''What is AMZN open,close,high,low data for the past month? 
Present the data with multiple columns for display in markdown.''')
Markdown(response.text)

Here is the open, close, high, and low data for AMZN stock for the past month, from approximately October 24, 2025, to November 21, 2025. Please note that stock markets are typically closed on weekends and holidays.

| Date       | Open      | High      | Low       | Close     |
| :--------- | :-------- | :-------- | :-------- | :-------- |
| Nov 21, 2025 | $216.35   | $222.21   | $215.18   | $220.69   |
| Nov 20, 2025 | $227.05   | $227.41   | $216.74   | $217.14   |
| Nov 19, 2025 | $223.74   | $223.74   | $218.52   | $222.69   |
| Nov 18, 2025 | $228.10   | $230.20   | $222.42   | $222.55   |
| Nov 17, 2025 | $233.25   | $234.60   | $229.19   | $232.87   |
| Nov 14, 2025 | $235.06   | $238.73   | $232.89   | $234.69   |
| Nov 13, 2025 | $243.05   | $243.75   | $236.50   | $237.58   |
| Nov 12, 2025 | $250.24   | $250.37   | $243.75   | $244.20   |
| Nov 11, 2025 | $248.41   | $249.75   | $247.23   | $249.10   |
| Nov 10, 2025 | $248.34   | $251.75   | $245.59   | $248.40   |
| Nov 07, 2025 | $242.90   | $244.90   | $238.49   | $244.41   |
| Nov 06, 2025 | $249.16   | $250.38   | $242.17   | $243.04   |
| Nov 05, 2025 | $249.03   | $251.00   | $246.16   | $250.20   |
| Nov 04, 2025 | $250.38   | $257.01   | $248.66   | $249.32   |
| Nov 03, 2025 | $255.36   | $258.60   | $252.90   | $254.00   |
| Oct 31, 2025 | $250.10   | $250.50   | $243.98   | $244.22   |
| Oct 30, 2025 | $227.06   | $228.44   | $222.75   | $222.86   |
| Oct 29, 2025 | $231.67   | $232.82   | $227.76   | $230.30   |
| Oct 28, 2025 | $228.00   | $230.00   | $225.00   | $227.00   |
| Oct 27, 2025 | $225.00   | $228.00   | $224.00   | $226.00   |
| Oct 24, 2025 | $221.97   | $225.40   | $221.90   | $224.21   |
| Oct 23, 2025 | $218.95   | $221.30   | $218.18   | $221.09   |
| Oct 22, 2025 | $219.30   | $220.01   | $216.52   | $217.95   |
| Oct 21, 2025 | $218.43   | $223.32   | $218.00   | $222.03   |
| Oct 20, 2025 | $213.88   | $216.69   | $213.59   | $216.48   |

# Previously on Kaggle: StockChat 1.0

## Validation BaseModels

In [11]:
# Validation BaseModels in pydantic schema.
class RestStatus(Enum):
    OK = "OK"
    DELAY = "DELAYED"
    NONE = "NOT_FOUND"
    AUTH = "NOT_AUTHORIZED"

class StopGeneration(BaseModel):
    result: str = Api.Const.Stop()

class RestResultPoly(BaseModel):
    request_id: Optional[str] = None
    count: Optional[int] = None
    next_url: Optional[str] = None
    status: RestStatus  

class MarketSession(Enum):
    PRE = "pre-market"
    REG = "regular"
    POST = "post-market"
    CLOSED = "closed"
    NA = "not applicable"

class MarketEvent(Enum):
    PRE_OPEN = 0
    REG_OPEN = 1
    REG_CLOSE = 2
    POST_CLOSE = 3
    LAST_CLOSE = 4

class AssetClass(Enum):
    STOCKS = "stocks"
    OPTION = "options"
    CRYPTO = "crypto"
    FOREX = "fx"
    INDEX = "indices"
    OTC = "otc"

class SymbolType(Enum):
    COMMON = "Common Stock"
    ETP = "ETP"
    ADR = "ADR"
    REIT = "REIT"
    DELISTED = ""
    CEF = "Closed-End Fund"
    UNIT = "Unit"
    RIGHT = "Right"
    EQUITY = "Equity WRT"
    GDR = "GDR"
    PREF = "Preference"
    CDI = "CDI"
    NVDR = "NVDR"
    REG = "NY Reg Shrs"
    MLP = "MLP"
    MUTUAL = "Mutual Fund"

class Locale(Enum):
    US = "us"
    GLOBAL = "global"

class Sentiment(Enum):
    V_POS = "very positive"
    POSITIVE = "positive"
    NEUTRAL_P = "neutral/positive"
    NEUTRAL_SP = "neutral/slightly positive"
    NEUTRAL = "neutral"
    NEUTRAL_SN = "neutral/slightly negative"
    NEUTRAL_N = "neutral/negative"
    MIXED = "mixed"
    NEGATIVE = "negative"
    V_NEG = "very negative"

class Trend(Enum):
    S_BUY = "strong-buy"
    BUY = "buy"
    HOLD = "hold"
    SELL = "sell"
    S_SELL = "strong-sell"

class MarketCondition(Enum):
    BULL = "bullish"
    BULLN = "cautiously bullish"
    HOLD = "hold"
    BEARN = "cautiously bearish"
    BEAR = "bearish"

class GeneratedEvent(BaseModel):
    last_close: str
    pre_open: str
    reg_open: str
    reg_close: str
    post_close: str
    timestamp: Optional[str] = None
    is_holiday: Optional[bool] = None

    def model_post_init(self, *args, **kwargs) -> None:
        if self.timestamp is None:
            self.timestamp = datetime.now(self.tz()).strftime('%c')
        if self.is_holiday is None:
            self.is_holiday = False

    def session(self, with_date: Optional[str] = None) -> MarketSession:
        if with_date is None:
            with_date = datetime.now(self.tz()).strftime('%c')
        compare = parse(with_date)
        if self.is_holiday or compare.weekday() > 4: # weekend
            return MarketSession.CLOSED
        events = [parse(event).time() for event in [self.pre_open,self.reg_open,self.reg_close,self.post_close]]
        if compare.time() < events[0]:
            return MarketSession.CLOSED
        else:
            session = MarketSession.NA
            if compare.time() >= events[0]:
                session = MarketSession.PRE
            if compare.time() >= events[1]:
                session = MarketSession.REG
            if compare.time() >= events[2]:
                session = MarketSession.POST
            if compare.time() >= events[3]:
                session = MarketSession.CLOSED
        return session

    def is_open(self) -> bool:
        return self.session() != MarketSession.CLOSED

    def has_update(self) -> bool:
        datetime_now = datetime.now(self.tz())
        self_ts = parse(self.timestamp)
        # Re-generate events for a new day.
        if datetime_now.day > self_ts.day:
            return True
        # No updates on holidays or when generated after post_close.
        if self.is_holiday or self_ts.time() >= parse(self.post_close).time():
            return False
        # Compare current time to generated event times.
        for event in [self.pre_open,self.reg_open,self.reg_close]:
            if datetime_now.time() > parse(event).time():
                return True
        # Current time is before pre_open.
        return False

    @classmethod
    def tz(cls):
        return pytz.timezone('US/Eastern') # Exchanges data is in eastern time.
    
    @classmethod
    def apply_fix(cls, value, fix: datetime) -> tuple[str, datetime]:
        api.validation_fail()
        value = fix.strftime('%c')
        return value, fix
    
    @field_validator("last_close")
    def valid_close(cls, value):
        date_gen = parse(value) # Generated close is in eastern time and tzinfo naive.
        date_now = parse(datetime.now(cls.tz()).strftime('%c')) # Need now in same format as generated.
        # Soft-pass: when actual session is closed after post-market
        if date_now.day == date_gen.day+1 and date_now.weekday() <= 4:
            date_fix = date_gen.replace(day=date_now.day)
            if date_fix.timestamp() < date_now.timestamp():
                value, date_gen = cls.apply_fix(value, date_fix) # soft-pass: use today's close
        # Soft-pass: when actual session is open post-market
        if date_now.day == date_gen.day and date_now.timestamp() < date_gen.timestamp():
            if date_now.weekday() > 0:
                date_fix = date_gen.replace(day=date_now.day-1)
            else:
                date_fix = date_gen.replace(day=date_now.day-3)
            if date_now.timestamp() > date_fix.timestamp():
                value, date_gen = cls.apply_fix(value, date_fix) # soft-pass: use previous close
        if date_now.weekday() == 0 or date_now.weekday() == 1 and date_gen.weekday() <= 4: # 0=monday, 4=friday
            return value # pass: generated thurs/friday on a monday/tues
        elif date_now.weekday() > 0 and date_now.weekday() <= 4 and date_gen.weekday() <= date_now.weekday()-1:
            return value # pass: generated yesterday/prior on a tues-fri
        elif date_now.weekday() > 4 and date_gen.weekday() <= 4:
            return value # pass: generated thurs/friday on a weekend
        elif date_now.day == date_gen.day and date_now.timestamp() > date_gen.timestamp():
            return value # pass: generated today after closed
        elif date_now.timestamp() < date_gen.timestamp():
            raise ValueError("last close cannot be a future value")
        else:
            raise ValueError("generated invalid last close")
        api.validation_fail()

class VectorStoreResult(BaseModel):
    docs: str
    dist: Optional[float] # requires query
    meta: Optional[dict]  # requires get or query
    store_id: str

class Aggregate(RestResultPoly):
    symbol: str
    open: float
    high: float
    low: float
    close: float
    volume: int
    otc: Optional[bool] = None
    preMarket: Optional[float] = None
    afterHours: Optional[float] = None

class DailyCandle(Aggregate):
    from_date: str

class AggregateWindow(BaseModel):
    o: float
    h: float
    l: float
    c: float
    v: int # traded volume
    n: Optional[int] = None # transaction count
    vw: Optional[float] = None # volume weighted average price
    otc: Optional[bool] = None
    t: int

    @field_validator("t")
    def valid_t(cls, value):
        if not value > 0:
            raise ValueError("invalid timestamp")
        if len(str(value)) == 13:
            return int(value/1000)
        return value

class CustomCandle(RestResultPoly): 
    ticker: str
    adjusted: bool
    queryCount: int
    resultsCount: int
    results: list[AggregateWindow]

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[AggregateWindow]:
        return self.results
    
class MarketStatus(BaseModel):
    exchange: str
    holiday: Optional[str] = None
    isOpen: bool
    session: Optional[MarketSession] = None
    t: int
    timezone: str

    def model_post_init(self, *args, **kwargs) -> None:
        if self.session is None:
            self.session = MarketSession.CLOSED
        if self.holiday is None:
            self.holiday = MarketSession.NA.value

class MarketStatusResult(BaseModel):
    results: MarketStatus

    def get(self) -> MarketStatus:
        return self.results

class Symbol(BaseModel):
    description: str
    displaySymbol: str
    symbol: str
    type: SymbolType

class SymbolResult(BaseModel):
    count: int
    result: list[Symbol]

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.result)

    def get(self) -> list[Symbol]:
        return self.result

class Quote(BaseModel):
    c: float
    d: float
    dp: float
    h: float
    l: float
    o: float
    pc: float
    t: int

    @field_validator("t")
    def valid_t(cls, value):
        if not value > 0:
            raise ValueError("invalid timestamp")
        return value

class PeersResult(BaseModel):
    results: list[str]
    count: Optional[int] = None

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[str]:
        return self.results

class BasicFinancials(BaseModel):
    metric: dict
    metricType: str
    series: dict
    symbol: str

class Insight(BaseModel):
    sentiment: Sentiment|MarketCondition
    sentiment_reasoning: str
    ticker: str

class Publisher(BaseModel):
    favicon_url: Optional[str]
    homepage_url: str
    logo_url: str
    name: str

class NewsSummary(BaseModel):
    title: str
    summary: Optional[str]
    insights: Optional[list[Insight]]
    published_utc: str

class NewsTypePoly(BaseModel):
    amp_url: Optional[str] = None
    article_url: str
    title: str
    author: str
    description: Optional[str] = None
    id: str
    image_url: Optional[str] = None
    insights: Optional[list[Insight]] = None
    keywords: Optional[list[str]] = None
    published_utc: str
    publisher: Publisher
    tickers: list[str]

    def summary(self):
        return NewsSummary(title=self.title,
                           summary=self.description,
                           insights=self.insights,
                           published_utc=self.published_utc)

class NewsResultPoly(RestResultPoly):
    results: list[NewsTypePoly]

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[NewsTypePoly]:
        return self.results

class NewsTypeFinn(BaseModel):
    category: str
    datetime: int
    headline: str
    id: int
    image: str
    related: str # symbol
    source: str
    summary: str
    url: str

    def summary(self):
        return NewsSummary(title=self.headline,
                           summary=self.summary,
                           insights=None,
                           published_utc=self.datetime)

class NewsResultFinn(BaseModel):
    results: list[NewsTypeFinn]
    count: Optional[int] = None

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[NewsTypeFinn]:
        return self.results

class NewsTypeGenerated(BaseModel):
    title: str
    summary: str
    insights: list[Insight]
    keywords: list[str]
    source: Publisher
    published_utc: str
    tickers: list[str]
    url: str

    def summary(self):
        return NewsSummary(title=self.title,
                           summary=self.summary,
                           insights=self.insights,
                           published_utc=self.published_utc)

class TickerOverview(BaseModel):
    ticker: str
    name: str
    market: AssetClass
    locale: Locale
    primary_exchange: Optional[str] = None
    active: bool
    currency_name: str
    cik: Optional[str] = None
    composite_figi: Optional[str] = None
    share_class_figi: Optional[str] = None
    market_cap: Optional[int|float] = None
    phone_number: Optional[str] = None
    address: Optional[dict] = None
    description: Optional[str] = None
    sic_code: Optional[str] = None
    sic_description: Optional[str] = None
    ticker_root: Optional[str] = None
    homepage_url: Optional[str] = None
    total_employees: Optional[int] = None
    list_date: Optional[str] = None
    branding: Optional[dict] = None
    share_class_shares_outstanding: Optional[int] = None
    weighted_shares_outstanding: Optional[int] = None
    round_lot: Optional[int] = None

class OverviewResult(RestResultPoly):
    results: TickerOverview

    def get(self) -> TickerOverview:
        return self.results

class RecommendationTrend(BaseModel):
    buy: int
    hold: int
    period: str
    sell: int
    strongBuy: int
    strongSell: int
    symbol: str

class TrendsResult(BaseModel):
    results: list[RecommendationTrend]
    count: Optional[int] = None

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[RecommendationTrend]:
        return self.results

## Contents Memory

In [12]:
# A contents-memory object.
class Memory:
    def __init__(self):
        self.system = f"""Give a concise, and detailed summary. Use information that you learn from the API responses.
        Use your tools and function calls according to the rules. Convert any all-upper case identifiers
        to proper case in your response. Convert any abbreviated or shortened identifiers to their full forms.
        Convert timestamps according to the rules before including them. Think step by step.
        """
        self.revery = {}
        self.contents = []
        self.prompt = None
        self.summary = None
        self.response = None
    
    def set_prompt(self, prompt):
        self.prompt = f"""
        The current date and time is: {datetime.now(GeneratedEvent.tz()).strftime('%c')}
        
        {prompt}
        """
        self.contents = [types.Content(role="user", parts=[types.Part(text=self.prompt)])]

    def set_reason(self, step):
        # Append the model's reasoning part.
        self.contents.append(types.Content(role="model", parts=[types.Part(thought=True,text=step)]))

    def append_code(self, prompt, code_response_parts):
        subroutine_content = [types.Content(role="user", parts=[types.Part(text=prompt)]),
                              types.Content(role="model", parts=code_response_parts)]
        # Append the model's generated code and execution result.
        self.revery[datetime.now(GeneratedEvent.tz()).strftime('%c')] = { 
            "contents": subroutine_content
        }

    def update_contents(self, function_call, api_response_part):
        # Append the model's function call part.
        self.contents.append(types.Content(role="model", parts=[types.Part(function_call=function_call)])) 
        # Append the api response part.
        self.contents.append(types.Content(role="user", parts=[api_response_part]))

    def set_summary(self, summary):
        self.summary = summary
        self.contents.append(types.Content(role="model", parts=[types.Part(text=summary)]))
        self.revery[datetime.now(GeneratedEvent.tz()).strftime('%c')] = {
            "prompt": self.prompt, 
            "summary": self.summary, 
            "contents": self.contents
        }
        self.contents = []

memory = Memory()

## Retrieval-Augmented Generation

In [13]:
# Define tool: retrieval-augmented generation.
# - using Chroma and text-embedding-004 for storage and retrieval
# - using gemini-2.0-flash for augmented generation
class RetrievalAugmentedGenerator:
    chroma_client = chromadb.PersistentClient(path="vector_db")
    config_temp = types.GenerateContentConfig(temperature=0.0)
    exchange_codes: Optional[dict] = None
    exchange_lists: dict = {}
    events: dict = {}
    holidays: dict = {}

    def __init__(self, genai_client, collection_name):
        self.client = genai_client
        self.embed_fn = GeminiEmbedFunction(genai_client)
        self.db = self.chroma_client.get_or_create_collection(
            name=collection_name, 
            embedding_function=self.embed_fn,  # type: ignore
            metadata={"hnsw:space": "cosine"})
        logging.getLogger("chromadb").setLevel(logging.ERROR) # suppress warning on existing id
        self.set_holidays("US", ["09-01-2025","10-13-2025","11-11-2025","11-27-2025","12-25-2025"])
        #self.generated_events("US")

    def set_holidays(self, exchange_code: str, holidays: list):
        self.holidays[exchange_code] = [datetime.strptime(h, "%m-%d-%Y").date() for h in holidays]

    def get_exchange_codes(self, with_query: Optional[str] = None):
        gen = None
        if with_query and with_query not in self.exchange_lists.keys():
            gen = tqdm(total=1, desc="Generate exchange codes with_query")
            data = self.get_exchanges_csv(
                f"""What is the {with_query} exchange code? Return only the exchange codes 
                as a list in string form. Just the list string. 
                Omit all other information or details. Do not chat or use sentences.""").candidates[0].content
            self.exchange_lists[with_query] = ast.literal_eval(data.parts[-1].text)
        elif with_query is None and self.exchange_codes is None:
            gen = tqdm(total=1, desc="Generate exchange codes")
            data = self.get_exchanges_csv(
                """Give me a dictionary in string form. It must contain key:value pairs 
                mapping exchange code to name. Just the dictionary string. 
                Omit all other information or details. Do not chat or use sentences.""").candidates[0].content
            self.exchange_codes = ast.literal_eval(data.parts[-1].text.strip("\\`"))
        if gen:
            gen.update(1)
        return self.exchange_lists[with_query] if with_query else self.exchange_codes

    def get_event_date(self, event_t: str, exchange_code: str, event: MarketEvent):
        current_dt_str = datetime.now(GeneratedEvent.tz()).strftime('%c')
        current_dt = datetime.strptime(current_dt_str, "%a %b %d %H:%M:%S %Y")
        current_t_str = datetime.now(GeneratedEvent.tz()).strftime('%H:%M:%S')
        current_t = datetime.strptime(current_t_str, "%H:%M:%S").time()
        event_time = parse(event_t).time()
        gen_datetime = None
        if event is MarketEvent.LAST_CLOSE:
            last_close_day = current_dt.date() - timedelta(days=0 if current_t > event_time else 1)
            # Loop backwards to find the last valid trading day (not a weekend or holiday).
            while last_close_day.weekday() >= 5 or last_close_day in self.holidays[exchange_code]: # 5 = Sat, 6 = Sun
                last_close_day -= timedelta(days=1)
            # Combine the date and time.
            gen_datetime = datetime.combine(last_close_day, event_time)
        else:
            next_event_day = current_dt.date() + timedelta(days=0 if current_t < event_time else 1)
            # Loop forward to find the next valid trading day (not a weekend or holiday).
            while next_event_day.weekday() >= 5 or next_event_day in self.holidays[exchange_code]: # 5 = Sat, 6 = Sun
                next_event_day += timedelta(days=1)
            # Combine date and time.
            gen_datetime = datetime.combine(next_event_day, event_time)
        # Format the result as requested.
        return gen_datetime.strftime('%a %b %d %X %Y')

    def generate_event(self, exchange_code: str, event: MarketEvent):
        if event is MarketEvent.LAST_CLOSE or event is MarketEvent.POST_CLOSE:
            prompt = f"""What is the closing time including post_market hours."""
        elif event is MarketEvent.PRE_OPEN or event is MarketEvent.REG_OPEN:
            is_pre = "including" if event is MarketEvent.PRE_OPEN else "excluding"
            prompt = f"""What is the opening time {is_pre} pre_market hours."""
        elif event is MarketEvent.REG_CLOSE:
            prompt = f"""What is the closing time excluding post_market hours."""
        prompt = f"""Answer based on your knowledge of exchange operating hours.
            Do not answer in full sentences. Omit all chat and provide the answer only.
            The fields pre_market and post_market both represent extended operating hours.

            The current date and time: {datetime.now(GeneratedEvent.tz()).strftime('%c')}
            
            Consider the {exchange_code} exchange's operating hours.
            {prompt}
            
            Answer with the time in this format: '%H:%M:%S'.
            Omit all other chat and details. Do not use sentences."""
        progress = tqdm(total=1, desc=f"Generate {exchange_code}->{event}")
        response = self.get_exchanges_csv(prompt).candidates[0].content
        try:
            if Api.Const.Stop() in f"{response.parts[-1].text}":
                self.generate_event_failed(progress, exchange_code, event)
            else:
                response = self.get_event_date(response.parts[-1].text, exchange_code, event)
                progress.update(1)
                return response
        except Exception as e:
            self.generate_event_failed(progress, exchange_code, event)

    def generate_event_failed(self, progress: tqdm, exchange_code: str, event: MarketEvent):
        progress.close()
        api.generation_fail()
        time.sleep(api.dt_between)
        return self.generate_event(exchange_code, event)

    def generated_events(self, exchange_code: str) -> GeneratedEvent:
        # Check for an existing GeneratedEvent object having updates.
        if exchange_code in self.events.keys() and self.events[exchange_code].has_update():
            event_obj = self.events[exchange_code]
            event_state = [(event_obj.pre_open, MarketEvent.PRE_OPEN),
                           (event_obj.reg_open, MarketEvent.REG_OPEN),
                           (event_obj.reg_close, MarketEvent.REG_CLOSE),
                           (event_obj.post_close, MarketEvent.POST_CLOSE)]
            # Need now in same format as generated.
            datetime_now = parse(datetime.now(event_obj.tz()).strftime('%c'))
            gen_ts = parse(event_obj.timestamp)
            # Re-generate events when day changes.
            if datetime_now.day > gen_ts.day:
                del self.events[exchange_code]
                return self.generated_events(exchange_code)
            # Update changed events on trading days.
            for e in event_state:
                if datetime_now > parse(e[0]):
                    event_dt = self.generate_event(exchange_code, e[1])
                    match e[1]:
                        case MarketEvent.PRE_OPEN:
                            event_obj.pre_open = event_dt
                        case MarketEvent.REG_OPEN:
                            event_obj.reg_open = event_dt
                        case MarketEvent.REG_CLOSE:
                            event_obj.reg_close = event_dt
                        case MarketEvent.POST_CLOSE:
                            event_obj.post_close = event_dt
            event_obj.timestamp = datetime.now(event_obj.tz()).strftime('%c')
            self.events[exchange_code] = event_obj
        # Generate events for an exchange code not in cache.
        elif exchange_code not in self.events.keys():
            self.events[exchange_code] = GeneratedEvent(
                last_close=self.generate_event(exchange_code, MarketEvent.LAST_CLOSE),
                pre_open=self.generate_event(exchange_code, MarketEvent.PRE_OPEN),
                reg_open=self.generate_event(exchange_code, MarketEvent.REG_OPEN),
                reg_close=self.generate_event(exchange_code, MarketEvent.REG_CLOSE),
                post_close=self.generate_event(exchange_code, MarketEvent.POST_CLOSE),
                is_holiday=datetime.now().date() in self.holidays[exchange_code])
        return self.events[exchange_code]

    def set_holiday_event(self, exchange_code: str):
        self.generated_events(exchange_code).is_holiday = True

    def last_market_close(self, exchange_code: str):
        return self.generated_events(exchange_code).last_close

    def add_documents_list(self, docs: list):
        self.embed_fn.document_mode = True # Switch to document mode.
        ids = list(map(str, range(self.db.count(), self.db.count()+len(docs))))
        metas=[{"source": doc.metadata["source"]} for doc in docs]
        content=[doc.page_content for doc in docs]
        tqdm(self.db.add(ids=ids, documents=content, metadatas=metas), desc="Generate document embedding")

    def add_api_document(self, query: str, api_response: str, topic: str, source: str = "add_api_document"):
        self.embed_fn.document_mode = True # Switch to document mode.
        splitter = RecursiveJsonSplitter(max_chunk_size=Api.Const.ChunkMax())
        docs = splitter.create_documents(texts=[api_response], convert_lists=True)
        ids = list(map(str, range(self.db.count(), self.db.count()+len(docs))))
        content = [json.dumps(doc.page_content) for doc in docs]
        metas = [{"source": source, "topic": topic}]*len(docs)
        tqdm(self.db.add(ids=ids, documents=content, metadatas=metas), desc="Generate api embedding")

    def add_peers_document(self, query: str, names: list, topic: str, source: str, group: str):
        self.embed_fn.document_mode = True # Switch to document mode.
        peers = {"symbol": topic, "peers": names}
        tqdm(self.db.add(ids=str(self.db.count()),
                         documents=[json.dumps(peers)],
                         metadatas=[{"source": source, "topic": topic, "group": group}]),
             desc="Generate peers embedding")

    def get_peers_document(self, query: str, topic: str, group: str):
        return self.get_documents_list(query, where={"$and": [{"group": group}, {"topic": topic}]})

    def add_rest_chunks(self, chunks: list, topic: str, source: str, ids: Optional[list[str]] = None,
                        meta_opt: Optional[list[dict]] = None, is_update: bool = True):
        self.embed_fn.document_mode = True # Switch to document mode
        if ids is None:
            ids = list(map(str, range(self.db.count(), self.db.count()+len(chunks))))
        if isinstance(chunks[0], BaseModel):
            docs = [model.model_dump_json() for model in chunks]
        else:
            docs = [json.dumps(obj) for obj in chunks]
        meta_base = {"source": source, "topic": topic}
        if meta_opt is not None:
            for m in meta_opt:
                m.update(meta_base)
        metas = [meta_base]*len(chunks) if meta_opt is None else meta_opt
        if is_update:
            tqdm(self.db.upsert(ids=ids, documents=docs, metadatas=metas), desc="Upsert chunks embedding")
        else:
            tqdm(self.db.add(ids=ids, documents=docs, metadatas=metas), desc="Add chunks embedding")

    def get_market_status(self, exchange_code: str) -> tuple[list[VectorStoreResult], bool]: # result, has rest update
        self.embed_fn.document_mode = False # Switch to query mode.
        stored = self.stored_result(self.db.get(where={
            "$and": [{"exchange": exchange_code}, {"topic": "market_status"}]}))
        if len(stored) == 0:
            return stored, True
        # Check for a daily market status update.
        status = json.loads(stored[0].docs)
        gen_day = parse(self.generated_events(exchange_code).timestamp).day
        store_day = parse(stored[0].meta['timestamp']).day
        if status["holiday"] != MarketSession.NA.value and gen_day == store_day:
            return stored, False
        elif gen_day > store_day:
            return stored, True
        # Update with generated events to avoid rest api requests.
        status["session"] = self.generated_events(exchange_code).session().value
        status["isOpen"] = self.generated_events(exchange_code).is_open()
        stored[0].docs = json.dumps(status)
        return stored, False

    def get_basic_financials(self, query: str, topic: str, source: str = "get_financials_1"):
        return self.get_documents_list(
            query, max_sources=200, where={"$and": [{"source": source}, {"topic": topic}]})

    def add_quote_document(self, query: str, quote: str, topic: str, timestamp: int, source: str):
        self.embed_fn.document_mode = True # Switch to document mode.
        tqdm(self.db.add(ids=str(self.db.count()), 
                             documents=[quote], 
                             metadatas=[{"source": source, "topic": topic, "timestamp": timestamp}]), 
             desc="Generate quote embedding")

    def get_api_documents(self, query: str, topic: str, source: str = "add_api_document", 
                          meta_opt: Optional[list[dict]] = None):
        where = [{"source": source}, {"topic": topic}]
        if meta_opt is None:
            return self.get_documents_list(query, where={"$and": where})
        else:
            for meta in meta_opt:
                for k,v in meta.items():
                    where.append({k: v})
            return self.get_documents_list(query, where={"$and": where})

    def query_api_documents(self, query: str, topic: str, source: str = "add_api_document"):
        return self.generate_answer(query, where={"$and": [{"source": source}, {"topic": topic}]})

    def add_grounded_document(self, query: str, topic: str, result):
        self.embed_fn.document_mode = True # Switch to document mode.
        chunks = result.candidates[0].grounding_metadata.grounding_chunks
        supports = result.candidates[0].grounding_metadata.grounding_supports
        if supports is not None: # Only add grounded documents which have supports
            grounded_text = [f"{s.segment.text}" for s in supports]
            source = [f"{c.web.title}" for c in chunks]
            score = [f"{s.confidence_scores}" for s in supports]
            tqdm(self.db.add(ids=str(self.db.count()),
                             documents=json.dumps(grounded_text),
                             metadatas=[{"source": ", ".join(source),
                                         "confidence_score": ", ".join(score),
                                         "topic": topic,
                                         "question": query}]),
                 desc="Generate grounding embedding")

    def get_grounding_documents(self, query: str, topic: str):
        self.embed_fn.document_mode = False # Switch to query mode.
        return self.stored_result(self.db.get(where={"$and": [{"question": query}, {"topic": topic}]}))
            
    def add_wiki_documents(self, title: str, wiki_chunks: list):
        self.embed_fn.document_mode = True # Switch to document mode.
        result = self.get_wiki_documents(title)
        if len(result) == 0:
            ids = list(map(str, range(self.db.count(), self.db.count()+len(wiki_chunks))))
            metas=[{"title": title, "source": "add_wiki_documents"}]*len(wiki_chunks)
            tqdm(self.db.add(ids=ids, documents=wiki_chunks, metadatas=metas), desc="Generate wiki embeddings")

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def generate_with_wiki_passages(self, query: str, title: str, passages: list[str]):
        return self.generate_answer(query, where={"title": title}, passages=passages)
    
    def get_wiki_documents(self, title: Optional[str] = None):
        self.embed_fn.document_mode = False # Switch to query mode.
        if title is None:
            return self.stored_result(self.db.get(where={"source": "add_wiki_document"}))
        else:
            return self.stored_result(self.db.get(where={"title": title}))

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_documents_list(self, query: str, max_sources: int = 5000, where: Optional[dict] = None):
        self.embed_fn.document_mode = False # Switch to query mode.
        return self.stored_result(
            self.db.query(query_texts=[query], 
                          n_results=max_sources, 
                          where=where), 
            is_query = True)

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_exchanges_csv(self, query: str):
        return self.generate_answer(query, max_sources=100, where={"source": "exchanges.csv"})

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def generate_answer(self, query: str, max_sources: int = 10, 
                        where: Optional[dict] = None, passages: Optional[list[str]] = None):
        stored = self.get_documents_list(query, max_sources, where)
        query_oneline = query.replace("\n", " ")
        prompt = f"""You're an expert writer. You understand how to interpret html and markdown. You will accept the
        question below and answer based only on the passages. Never mention the passages in your answers. Be sure to 
        respond in concise sentences. Include all relevant background information when possible. If a passage is not 
        relevant to the answer you must ignore it. If no passage answers the question respond with: I don't know.

        QUESTION: {query_oneline}
        
        """
        # Add the retrieved documents to the prompt.
        stored_docs = [passage.docs for passage in stored]
        for passage in stored_docs if passages is None else stored_docs + passages:
            passage_oneline = passage.replace("\n", " ")
            prompt += f"PASSAGE: {passage_oneline}\n"
        # Generate the response.
        response = api.retriable(
            self.client.models.generate_content,
            model=api(Api.Model.GEN),
            config=self.config_temp,
            contents=prompt)
        # Check for generated code and store in memory.
        content = response.candidates[0].content
        if len(content.parts) > 1 and content.parts[0].executable_code:
            memory.append_code(prompt, content.parts)
        return response

    def stored_result(self, result, is_query: bool = False) -> list[VectorStoreResult]:
        try:
            results = []
            if len(result["documents"]) == 0:
                return results
            if isinstance(result["documents"][0], list):
                for i in range(len(result["documents"][0])):
                    obj = VectorStoreResult(docs=result["documents"][0][i],
                                            dist=result["distances"][0][i] if is_query else None,
                                            meta=result["metadatas"][0][i],
                                            store_id=result["ids"][0][i])
                    results.append(obj)
            else:
                results.append(
                    VectorStoreResult(docs=result["documents"][0],
                                      dist=result["distances"][0] if is_query else None,
                                      meta=result["metadatas"][0],
                                      store_id=result["ids"][0]))
            return results
        except Exception as e:
            raise e

## Wiki Grounding

In [14]:
# Define tool: wiki-grounding generation.
# - using gemini-2.0-flash for response generation
# - using a RAG-implementation to store groundings
# - create new groundings by similarity to topic
# - retrieve existing groundings by similarity to topic
class WikiGroundingGenerator:   
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl
        with warnings.catch_warnings():
            warnings.simplefilter("ignore") # suppress beta-warning
            self.splitter = HTMLSemanticPreservingSplitter(
                headers_to_split_on=[("h2", "Main Topic"), ("h3", "Sub Topic")],
                separators=["\n\n", "\n", ". ", "! ", "? "],
                max_chunk_size=Api.Const.ChunkMax(),
                chunk_overlap=50,
                preserve_links=True,
                preserve_images=True,
                preserve_videos=True,
                preserve_audio=True,
                elements_to_preserve=["table", "ul", "ol", "code"],
                denylist_tags=["script", "style", "head"],
                custom_handlers={"code": self.code_handler},
            )

    def generate_answer(self, query: str, topic: str):
        stored = self.rag.get_wiki_documents(topic)
        if len(stored) > 0:
            return self.rag.generate_with_wiki_passages(query, topic, [chunk.docs for chunk in stored]).text
        else:
            pages = wikipedia.search(topic + " company")
            if len(pages) > 0:
                p_topic_match = 0.80
                for i in range(len(pages)):
                    if tqdm(api.similarity([topic + " company", pages[i]]) > p_topic_match, 
                            desc= "Score wiki search by similarity to topic"):
                        page_html = Api.get(f"https://en.wikipedia.org/wiki/{pages[i]}")
                        chunks = [chunk.page_content for chunk in self.splitter.split_text(page_html)]
                        self.rag.add_wiki_documents(topic, chunks)
                        return self.rag.generate_with_wiki_passages(query, topic, chunks).text
            return Api.Const.Stop()

    def code_handler(self, element: Tag) -> str:
        data_lang = element.get("data-lang")
        code_format = f"<code:{data_lang}>{element.get_text()}</code>"
        return code_format

## Search Grounding

In [15]:
# Define tool: search-grounding generation.
# - using gemini-2.0-flash with GoogleSearch tool for response generation
# - using a RAG-implementation to store groundings
# - create new groundings by exact match to topic
# - retrieve existing groundings by similarity to topic
class SearchGroundingGenerator:
    config_ground = types.GenerateContentConfig(
        tools=[types.Tool(google_search=types.GoogleSearch())],
        temperature=0.0
    )
    
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl

    def generate_answer(self, query: str, topic: str):
        stored = self.rag.get_grounding_documents(query, topic)
        if len(stored) > 0:
            for i in range(len(stored)):
                meta_q = stored[i].meta["question"]
                p_ground_match = 0.95 # This can be really high ~ 95-97%
                if tqdm(api.similarity([query, meta_q]) > p_ground_match,
                        desc="Score similarity to stored grounding"):
                    return ast.literal_eval(stored[i].docs)
        return self.get_grounding(query, topic)

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_grounding(self, query: str, topic: str):
        contents = [types.Content(role="user", parts=[types.Part(text=query)])]
        contents += f"""
        You're a search assistant that provides answers to questions about {topic}.
        Do not discuss alternative topics of interest. Do not discuss similar topics.
        You will provide answers that discuss only {topic}. 
        You may discuss the owner or parent of {topic} when no other answer is possible.
        Otherwise respond with: I don't know."""
        response = api.retriable(self.client.models.generate_content, 
                                 model=api(Api.Model.GEN), 
                                 config=self.config_ground, 
                                 contents=contents)
        if response.candidates[0].grounding_metadata.grounding_supports is not None:
            if self.is_consistent(query, topic, response.text):
                self.rag.add_grounded_document(query, topic, response)
                return response.text 
        return Api.Const.Stop() # Empty grounding supports or not consistent in response

    def is_consistent(self, query: str, topic: str, model_response: str) -> bool:
        topic = topic.replace("'", "")
        id_strs = topic.split()
        if len(id_strs) == 1:
            matches = re.findall(rf"{id_strs[0]}[\s,.]+\S+", query)
            if len(matches) > 0:
                topic = matches[0]
        compound_match = re.findall(rf"{id_strs[0]}[\s,.]+\S+", model_response)
        model_response = model_response.replace("'", "")
        if len(compound_match) == 0 and topic in model_response:
            return True # not a compound topic id and exact topic match
        for match in compound_match:
            if topic not in match:
                return False
        return True # all prefix matches contained topic

## Rest Grounding

In [16]:
# Rest api-helpers to manage request-per-minute limits.
# - define an entry for each endpoint limit
# - init rest tool with limits to create blocking queues
# - apply a limit to requests with rest_tool.try_url
class ApiLimit(Enum):
    FINN = "finnhub.io",50
    POLY = "polygon.io",4 # (id_url,rpm)

class BlockingUrlQueue:
    on_cooldown = False
    cooldown = None
    cooldown_start = None
    
    def __init__(self, rest_fn: Callable, per_minute: int):
        self.per_minute_max = per_minute
        self.quota = per_minute
        self.rest_fn = rest_fn

    def push(self, rest_url: str):
        if not self.on_cooldown:
            self.cooldown = Timer(60, self.reset_quota)
            self.cooldown.start()
            self.cooldown_start = time.time()
            self.on_cooldown = True
        if self.quota > 0:
            self.quota -= 1
            time.sleep(0.034) # ~30 requests per second
            return self.rest_fn(rest_url)
        else:
            print(f"limited {self.per_minute_max}/min, waiting {self.limit_expiry()}s")
            time.sleep(max(self.limit_expiry(),0.5))
            return self.push(rest_url)

    def reset_quota(self):
        self.quota = self.per_minute_max
        self.on_cooldown = False
        self.cooldown_start = None

    def limit_expiry(self):
        if self.cooldown_start:
            return max(60-(time.time()-self.cooldown_start),0)
        return 0

In [17]:
# Define tool: rest-grounding generation.
# - using gemini-2.0-flash for response generation
# - using a RAG-implementation to store groundings
# - reduce long-context by chunked pre-processing
class RestGroundingGenerator:    
    limits = None

    def __init__(self, rag_impl, with_limits: bool):
        self.rag = rag_impl
        if with_limits:
            self.limits = {}
            for rest_api in ApiLimit:
                self.limits[rest_api.value[0]] = BlockingUrlQueue(Api.get, rest_api.value[1])

    def get_limit(self, rest_api: ApiLimit) -> Optional[BlockingUrlQueue]:
        return self.limits[rest_api.value[0]] if self.limits else None

    def basemodel(self, data: str, schema: BaseModel, from_lambda: bool = False) -> Optional[BaseModel]:
        try:
            if from_lambda:
                return schema(results=json.loads(data))
            return schema.model_validate_json(data)
        except Exception as e:
            raise e

    def dailycandle(self, data: str) -> Optional[DailyCandle]:
        try:
            candle = json.loads(data)
            if "from" not in candle:
                raise ValueError("not a dailycandle / missing value for date")
            agg = self.basemodel(data, Aggregate)
            return DailyCandle(from_date=candle["from"], 
                               status=agg.status.value, 
                               symbol=agg.symbol, 
                               open=agg.open, 
                               high=agg.high, 
                               low=agg.low, 
                               close=agg.close, 
                               volume=agg.volume, 
                               otc=agg.otc, 
                               preMarket=agg.preMarket, 
                               afterHours=agg.afterHours)
        except Exception as e:
            raise e

    @retry.Retry(timeout=600)
    def try_url(self, url: str, schema: BaseModel, as_lambda: bool, with_limit: Optional[BlockingUrlQueue],
                success_fn: Callable, *args, **kwargs):
        try:
            if self.limits is None:
                data = Api.get(url)
            elif with_limit:
                data = with_limit.push(url)
            if schema is DailyCandle:
                model = self.dailycandle(data)
            else:
                model = self.basemodel(data, schema, as_lambda)
        except Exception as e:
            try:
                print(f"try_url exception: {e}")
                if issubclass(schema, RestResultPoly):
                    return success_fn(*args, **kwargs, result=self.basemodel(data, RestResultPoly))
            except Exception as not_a_result:
                print(not_a_result)
            return StopGeneration()
        else:
            return success_fn(*args, **kwargs, model=model)

    def get_symbol_matches(self, with_content, by_name: bool, model: SymbolResult):
        matches = []
        max_failed_match = model.count if not by_name else 3
        p_desc_match = 0.92
        p_symb_match = 0.95
        if model.count > 0:
            for obj in tqdm(model.get(), desc="Score similarity to query"):
                if max_failed_match > 0:
                    desc = [with_content["q"].upper(), obj.description.split("-", -1)[0]]
                    symb = [with_content["q"].upper(), obj.symbol]
                    if by_name and api.similarity(desc) > p_desc_match: 
                        matches.append(obj.symbol)
                    elif not by_name and api.similarity(symb) > p_symb_match:
                        matches.append(obj.description)
                        max_failed_match = 0
                    else:
                        max_failed_match -= 1
        if len(matches) > 0:
            self.rag.add_api_document(with_content["query"], matches, with_content["q"], "get_symbol_1")
            return matches
        return Api.Const.Stop()

    def get_quote(self, with_content, model: Quote):
        quote = model.model_dump_json()
        self.rag.add_quote_document(with_content["query"], quote, with_content["symbol"], model.t, "get_quote_1")
        return quote

    def parse_financials(self, with_content, model: BasicFinancials):
        metric = list(model.metric.items())
        chunks = []
        # Chunk the metric data.
        for i in range(0, len(metric), Api.Const.MetricBatch()):
            batch = metric[i:i + Api.Const.MetricBatch()]
            chunks.append({"question": with_content["query"], "answer": batch})
        # Chunk the series data.
        for key in model.series.keys():
            series = list(model.series[key].items())
            for s in series:
                if api.token_count(s) <= Api.Const.ChunkMax():
                    chunks.append({"question": with_content["query"], "answer": s})
                else:
                    k = s[0]
                    v = s[1]
                    for i in range(0, len(v), Api.Const.SeriesBatch()):
                        batch = v[i:i + Api.Const.SeriesBatch()]
                        chunks.append({"question": with_content["query"], "answer": {k: batch}})
        self.rag.add_rest_chunks(chunks, topic=with_content["symbol"], source="get_financials_1")
        return chunks

    def parse_news(self, with_content, model: NewsResultFinn):
        if model.count > 0:
            metas = []
            for digest in model.get():
                pub_date = datetime.fromtimestamp(digest.datetime, tz=GeneratedEvent.tz()).strftime("%Y-%m-%d")
                metas.append({"publisher": digest.source,
                              "published_est": parse(pub_date).timestamp(),
                              "news_id": digest.id,
                              "related": digest.related})
            self.rag.add_rest_chunks(model.get(), topic=with_content["symbol"], source="get_news_1",
                                     ids=[f"{digest.id}+news" for digest in model.get()],
                                     meta_opt=metas, is_update=False)
            return [digest.summary().model_dump_json() for digest in model.get()]
        return Api.Const.Stop()

    def parse_news(self, with_content, model: Optional[NewsResultPoly] = None,
                   result: Optional[RestResultPoly] = None) -> tuple[list, str]: # list of summary, next list url
        if model and model.status in [RestStatus.OK, RestStatus.DELAY]:
            metas = []
            for news in model.get():
                pub_date = parse(news.published_utc).strftime("%Y-%m-%d")
                metas.append({"publisher": news.publisher.name,
                              "published_utc": parse(pub_date).timestamp(),
                              "news_id": news.id,
                              "related": json.dumps(news.tickers),
                              "keywords": json.dumps(news.keywords)})
            self.rag.add_rest_chunks(model.get(), topic=with_content["ticker"], source="get_news_2",
                                     ids=[news.id for news in model.get()],
                                     meta_opt=metas, is_update=False)
            return [news.summary().model_dump_json() for news in model.get()], model.next_url
        elif result:
            return result.model_dump_json()

    def parse_daily_candle(self, with_content, model: Optional[DailyCandle] = None,
                           result: Optional[RestResultPoly] = None):
        if model and model.status in [RestStatus.OK, RestStatus.DELAY]:
            self.rag.add_rest_chunks(
                chunks=[model],
                topic=with_content["stocksTicker"],
                source="daily_candle_2",
                meta_opt=[{"from_date": model.from_date, "adjusted": with_content["adjusted"]}])
            return model
        elif result:
            return result

    def parse_custom_candle(self, with_content, model: Optional[CustomCandle] = None,
                            result: Optional[RestResultPoly] = None):
        if model and model.status in [RestStatus.OK, RestStatus.DELAY]:
            metas = [{
                "timespan": with_content["timespan"],
                "adjusted": with_content["adjusted"],
                "from": with_content["from"],
                "to": with_content["to"]}]*model.count
            candles = [candle.model_dump_json() for candle in model.get()]
            self.rag.add_rest_chunks(
                chunks=candles,
                topic=with_content["stocksTicker"],
                source="custom_candle_2",
                meta_opt=metas)
            return candles
        elif result:
            return result.model_dump_json()

    def parse_overview(self, with_content, model: OverviewResult):
        overview = [model.get().model_dump_json()]
        self.rag.add_rest_chunks(chunks=overview, topic=with_content["ticker"], source="ticker_overview_2")
        return overview

    def parse_trends(self, with_content, model: TrendsResult):
        if model.count > 0:
            metas = [{"period": trend.period} for trend in model.get()]
            trends = [trend.model_dump_json() for trend in model.get()]
            self.rag.add_rest_chunks(trends, topic=with_content["symbol"], source="trends_1", meta_opt=metas)
            return trends
        return Api.Const.Stop()

    def augment_market_status(self, with_id: Optional[str], model: MarketStatusResult):
        if model.get().holiday != MarketSession.NA.value:
            self.rag.set_holiday_event(model.get().exchange)
        events = self.rag.generated_events(model.get().exchange)
        model.get().session = events.session()
        model.get().isOpen = events.is_open()
        meta = {"exchange": model.get().exchange,
                "last_close": events.last_close,
                "pre_open": events.pre_open,
                "reg_open": events.reg_open,
                "reg_close": events.reg_close,
                "post_close": events.post_close,
                "timestamp": events.timestamp }
        self.rag.add_rest_chunks([model.get()],
                                 topic="market_status",
                                 source="get_market_status_1",
                                 ids=[with_id] if with_id else None,
                                 meta_opt=[meta])
        return model.get().model_dump_json()

    def get_symbol(self, content, by_name: bool = True):
        return self.try_url(
            f"https://finnhub.io/api/v1/search?q={content['q']}&exchange={content['exchange']}&token={FINNHUB_API_KEY}",
            schema=SymbolResult,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.get_symbol_matches,
            with_content=content,
            by_name=by_name)

    def get_current_price(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/quote?symbol={content['symbol']}&token={FINNHUB_API_KEY}",
            schema=Quote,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.get_quote,
            with_content=content)

    def get_market_status(self, content, store_id: Optional[str] = None):
        return self.try_url(
            f"https://finnhub.io/api/v1/stock/market-status?exchange={content['exchange']}&token={FINNHUB_API_KEY}",
            schema=MarketStatusResult,
            as_lambda=True,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.augment_market_status,
            with_id=store_id)

    def get_peers(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/stock/peers?symbol={content['symbol']}&grouping={content['grouping']}&token={FINNHUB_API_KEY}",
            schema=PeersResult,
            as_lambda=True,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=lambda model: model)

    def get_basic_financials(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/stock/metric?symbol={content['symbol']}&metric={content['metric']}&token={FINNHUB_API_KEY}",
            schema=BasicFinancials,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.parse_financials,
            with_content=content)

    def get_news_simple(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/company-news?symbol={content['symbol']}&from={content['from']}&to={content['to']}&token={FINNHUB_API_KEY}",
            schema=NewsResultFinn,
            as_lambda=True,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.parse_news,
            with_content=content)

    def get_news_tagged(self, content):
        next_url = f"https://api.polygon.io/v2/reference/news?ticker={content['ticker']}&published_utc.gte={content['published_utc.gte']}&published_utc.lte={content['published_utc.lte']}&order={content['order']}&limit={content['limit']}&sort={content['sort']}&apiKey={POLYGON_API_KEY}"
        news = []
        while True:
            news_list, next_url = self.try_url(
                next_url,
                schema=NewsResultPoly,
                as_lambda=False,
                with_limit=self.get_limit(ApiLimit.POLY),
                success_fn=self.parse_news,
                with_content=content)
            news += news_list
            if next_url is None:
                break
            next_url += f"&apiKey={POLYGON_API_KEY}"
        return news

    def get_daily_candle(self, content):
        return self.try_url(
            f"https://api.polygon.io/v1/open-close/{content['stocksTicker']}/{content['date']}?adjusted={content['adjusted']}&apiKey={POLYGON_API_KEY}",
            schema=DailyCandle,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.POLY),
            success_fn=self.parse_daily_candle,
            with_content=content)

    def get_custom_candle(self, content):
        return self.try_url(
            f"https://api.polygon.io/v2/aggs/ticker/{content['stocksTicker']}/range/{content['multiplier']}/{content['timespan']}/{content['from']}/{content['to']}?adjusted={content['adjusted']}&sort={content['sort']}&limit={content['limit']}&apiKey={POLYGON_API_KEY}",
            schema=CustomCandle,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.POLY),
            success_fn=self.parse_custom_candle,
            with_content=content)

    def get_overview(self, content):
        return self.try_url(
            f"https://api.polygon.io/v3/reference/tickers/{content['ticker']}?apiKey={POLYGON_API_KEY}",
            schema=OverviewResult,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.POLY),
            success_fn=self.parse_overview,
            with_content=content)

    def get_trends_simple(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/stock/recommendation?symbol={content['symbol']}&token={FINNHUB_API_KEY}",
            schema=TrendsResult,
            as_lambda=True,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.parse_trends,
            with_content=content)

## Callable Functions

In [18]:
# Callable functions in openapi schema.
decl_get_symbol_1 = types.FunctionDeclaration(
    name="get_symbol_1",
    description="""Search for the stock ticker symbol of a given company, security, isin or cusip. Each ticker
                   entry provides a description, symbol, and asset type. If this doesn't help you should try 
                   calling get_wiki_tool_response next.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": """A ticker symbol to search for."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["q", "exchange", "query"]
    }
)

decl_get_symbols_1 = types.FunctionDeclaration(
    name="get_symbols_1",
    description="""List all supported symbols and tickers. The results are filtered by exchange code.""",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter the results."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["exchange", "query"]
    }
)

decl_get_name_1 = types.FunctionDeclaration(
    name="get_name_1",
    description="""Search for the name associated with a stock ticker or symbol's company, security, isin or cusip. 
    Each ticker entry provides a description, matching symbol, and asset type.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": """The symbol or ticker to search for."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            },
            "company": {
                "type": "string",
                "description": "The company you're searching for."
            }
        },
        "required": ["q", "exchange", "query", "company"]
    }
)

decl_get_symbol_quote_1 = types.FunctionDeclaration(
    name="get_symbol_quote_1",
    description="""Search for the current price or quote of a stock ticker or symbol. The response is
                   provided in json format. Each response contains the following key-value pairs:
                   
                   c: Current price,
                   d: Change,
                  dp: Percent change,
                   h: High price of the day,
                   l: Low price of the day,
                   o: Open price of the day,
                  pc: Previous close price,
                   t: Epoch timestamp of price in seconds.

                   Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "The stock ticker symbol for a company, security, isin, or cusip." 
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            },
            "exchange": {
                "type": "string",
                "description": "The exchange code used to filter quotes. This must always be 'US'."
            }
        },
        "required": ["symbol", "query", "exchange"]
    }
)

decl_get_local_datetime = types.FunctionDeclaration(
    name="get_local_datetime",
    description="""Converts an array of timestamps from epoch time to the local timezone format. The result is an array
                   of date and time in locale appropriate format. Suitable for use in a locale appropriate response.
                   Treat this function as a vector function. Always prefer to batch timestamps for conversion. Use this
                   function to format date and time in your responses.""",
    parameters={
        "type": "object",
        "properties": {
            "t": {
                "type": "array",
                "description": """An array of timestamps in seconds since epoch to be converted. The order of
                                  timestamps matches the order of conversion.""",
                "items": {
                    "type": "integer"
                }
            }
        },
        "required": ["t"]
    }
)

decl_get_market_status_1 = types.FunctionDeclaration(
    name="get_market_status_1",
    description="""Get the current market status of global exchanges. Includes whether exchanges are open or closed.  
                   Also includes holiday details if applicable. The response is provided in json format. Each response 
                   contains the following key-value pairs:

                   exchange: Exchange code,
                   timezone: Timezone of the exchange,
                    holiday: Holiday event name, or null if it's not a holiday,
                     isOpen: Whether the market is open at the moment,
                          t: Epoch timestamp of status in seconds (Eastern Time),
                    session: The market session can be 1 of the following values: 
                    
                    pre-market,regular,post-market when open, or null if closed.
                    
                    Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            }
        },
        "required": ["exchange"]
    }
)

decl_get_market_session_1 = types.FunctionDeclaration(
    name="get_market_session_1",
    description="Get the current market session of global exchanges.",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            }
        },
        "required": ["exchange"]
    }
)

decl_get_company_peers_1 = types.FunctionDeclaration(
    name="get_company_peers_1",
    description="""Search for a company's peers. Returns a list of peers operating in the same country and in the same
                   sector, industry, or subIndustry. Each response contains the following key-value pairs: 
                   
                   symbol: The company's stock ticker symbol, 
                   peers: A list containing the peers.
                   
                   Each peers entry contains the following key-value pairs:
                   
                   symbol: The peer company's stock ticker symbol, 
                   name: The peer company's name.
                   
                   Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "The stock ticker symbol of a company to obtain peers."
            },
            "grouping": {
                "type": "string",
                "description": """This parameter may be one of the following values: sector, industry, subIndustry.
                                  Always use subIndustry unless told otherwise."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "grouping", "exchange", "query"]
    }
)

decl_get_exchange_codes_1 = types.FunctionDeclaration(
    name="get_exchange_codes_1",
    description="""Get a dictionary mapping all supported exchange codes to their names."""
)

decl_get_exchange_code_1 = types.FunctionDeclaration(
    name="get_exchange_code_1",
    description="""Search for the exchange code to use when filtering by exchange. The result will be one or
                   more exchange codes provided as a comma-separated string value.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": "Specifies which exchange code to search for."
            }
        },
        "required": ["q"]
    }
)

decl_get_financials_1 = types.FunctionDeclaration(
    name="get_financials_1",
    description="""Get company basic financials such as margin, P/E ratio, 52-week high/low, etc. Parse the response for 
                   key-value pairs in json format and interpret their meaning as stock market financial indicators.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "metric": {
                "type": "string",
                "description": "It must always be declared as the value 'all'"
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "metric", "query"]
    }
)

decl_get_daily_candlestick_2 = types.FunctionDeclaration(
    name="get_daily_candlestick_2",
    description="""Get a historical daily stock ticker candlestick / aggregate bar (OHLC). 
                   Includes historical daily open, high, low, and close prices. Also includes historical daily trade
                   volume and pre-market/after-hours trade prices. It provides the last trading days' data after 
                   11:59PM Eastern Time.""",
    parameters={
        "type": "object",
        "properties": {
            "stocksTicker": {
                "type": "string",
                "description": "The stock ticker symbol of a company to search for.",
            },
            "date": {
                "type": "string",
                "format": "date-time",
                "description": """The date of the requested candlestick in format YYYY-MM-DD."""
            },
            "adjusted": {
                "type": "string",
                "description": """May be true or false. Indicates if the results should be adjusted for splits.
                                  Use true unless told otherwise."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["stocksTicker", "date", "adjusted", "exchange", "query"]
    },
)

decl_get_company_news_1 = types.FunctionDeclaration(
    name="get_company_news_1",
    description="Retrieve the most recent news articles related to a specified ticker.",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company.",
            },
            "from": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD. It must be older than the parameter 'to'."""
            },
            "to": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD. It must be more recent than the parameter 'from'. The
                                  default value is today's date."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "from", "to", "query"]
    },
)

decl_get_custom_candlestick_2 = types.FunctionDeclaration(
    name="get_custom_candlestick_2",
    description="""Get a historical stock ticker candlestick / aggregate bar (OHLC) over a custom date range and 
                   time interval in Eastern Time. Includes historical open, high, low, and close prices. Also 
                   includes historical daily trade volume and pre-market/after-hours trade prices. It includes 
                   the last trading days' data after 11:59PM Eastern Time.""",
    parameters={
        "type": "object",
        "properties": {
            "stocksTicker": {
                "type": "string",
                "description": "The stock ticker symbol of a company to search for.",
            },
            "multiplier": {
                "type": "integer",
                "description": "This must be included and equal to 1 unless told otherwise."
            },
            "timespan": {
                "type": "string",
                "description": """The size of the candlestick's time window. This is allowed to be one of the following:
                                  second, minute, hour, day, week, month, quarter, or year. The default value is day."""
            },
            "from": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be older than the parameter 'to'."""
            },
            "to": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be more recent than the parameter 'from'. The 
                                  default is one weekday before get_last_market_close.
                                  Replace more recent dates with the default."""
            },
            "adjusted": {
                "type": "string",
                "description": """May be true or false. Indicates if the results should be adjusted for splits.
                                  Use true unless told otherwise."""
            },
            "sort": {
                "type": "string",
                "description": """This must be included. May be one of asc or desc. asc will sort by timestmap in 
                                  ascending order. desc will sort by timestamp in descending order."""
            },
            "limit": {
                "type": "integer",
                "description": """Set the number of base aggregates used to create this candlestick. This must be 5000 
                                  unless told to limit base aggregates to something else."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["stocksTicker", "multiplier", "timespan", "from", "to", "adjusted", "sort", "limit", "query"]
    },
)

decl_get_last_market_close = types.FunctionDeclaration(
    name="get_last_market_close",
    description="""Get the last market close of the specified exchange in Eastern Time. The response has already
                   been converted by get_local_datetime so this step should be skipped.""",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            }
        },
        "required": ["exchange"]
    }
)

decl_get_ticker_overview_2 = types.FunctionDeclaration(
    name="get_ticker_overview_2",
    description="""Retrieve comprehensive details for a single ticker symbol. It's a deep look into a company’s 
    fundamental attributes, including its primary exchange, standardized identifiers (CIK, composite FIGI, 
    share class FIGI), market capitalization, industry classification, and key dates. Also includes branding assets in
    the form of icons and logos.
    """,
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker symbol of a company."
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["ticker", "query"]
    }
)

decl_get_recommendation_trends_1 = types.FunctionDeclaration(
    name="get_recommendation_trends_1",
    description="""Get the latest analyst recommendation trends for a company.
                The data includes the latest recommendations as well as historical
                recommendation data for each month. The data is classified according
                to these categories: strongBuy, buy, hold, sell, and strongSell.
                The date of a recommendation indicated by the value of 'period'.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "query"]
    }
)

decl_get_news_with_sentiment_2 = types.FunctionDeclaration(
    name="get_news_with_sentiment_2",
    description="""Retrieve the most recent news articles related to a specified ticker. Each article includes 
                   comprehensive coverage. Including a summary, publisher information, article metadata, 
                   and sentiment analysis.""",
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "published_utc.gte": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be older than the parameter 'published_utc.lte'. 
                                  The default value is one-month ago from today's date."""
            },
            "published_utc.lte": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be more recent than the parameter 'published_utc.gte'.
                                  The default is one weekday prior to get_last_market_close (excluding weekends).
                                  Replace more recent dates with the default."""
            },
            "order": {
                "type": "string",
                "description": """Must be desc for descending order, or asc for ascending order.
                                  When order is not specified the default is descending order.
                                  Ordering will be based on the parameter 'sort'."""
            },
            "limit": {
                "type": "integer",
                "description": """This must be included and equal to 1000 unless told otherwise."""
            },
            "sort": {
                "type": "string",
                "description": """The sort field used for ordering. This value must
                                  always be published_utc."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["limit", "ticker", "published_utc.gte", "published_utc.lte", "order", "sort", "query"]
    }
)

decl_get_rag_tool_response = types.FunctionDeclaration(
    name="get_rag_tool_response",
    description="""A database containing useful financial information. Always check here for answers first.""",
    parameters={
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "A question needing an answer. Asked as a simple string."
            }
        }
    }
)

decl_get_wiki_tool_response = types.FunctionDeclaration(
    name="get_wiki_tool_response",
    description="""Answers questions that still have unknown answers. Retrieve a wiki page related to a company, 
                   product, or service. Each web page includes detailed company information, financial indicators, 
                   tickers, symbols, history, and products and services.""",
    parameters={
        "type": "object",
        "properties": {
            "id": {
                "type": "string",
                "description": "The question's company or product. Just the name and no other details."
            },
            "q": {
                "type": "string",
                "description": "The complete, unaltered, query string."
            }
        },
        "required": ["id", "q"]
    }
)

decl_get_search_tool_response = types.FunctionDeclaration(
    name="get_search_tool_response",
    description="Answers questions that still have unknown answers. Use it after checking all your other tools.",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": "The question needing an answer. Asked as a simple string."
            },
            "id": {
                "type": "string",
                "description": "The question's company or product. In one word. Just the name and no other details."
            }
        },
        "required": ["q", "id"]
    }
)

In [19]:
# Import the finance api secret keys.

POLYGON_API_KEY = UserSecretsClient().get_secret("POLYGON_API_KEY")
FINNHUB_API_KEY = UserSecretsClient().get_secret("FINNHUB_API_KEY")

In [20]:
# Instantiate tools and load the exchange data from source csv.
# - Identifies exchanges by a 1-2 letter code which can be used to filter response data.
# - Also maps the exchange code to exchange details.
try:
    df = pandas.read_csv("/kaggle/input/exchanges/exchanges_src.csv")
except FileNotFoundError as e:
    df = pandas.read_csv("exchanges_src.csv") # local run
df = df.drop(["close_date"], axis=1).fillna("")
df.to_csv("exchanges.csv", index=False)
exchanges = CSVLoader(file_path="exchanges.csv", encoding="utf-8", csv_args={"delimiter": ","}).load()

# Prepare a RAG tool for use and add the exchange data.
tool_rag = RetrievalAugmentedGenerator(api.args.CLIENT, "finance")
tool_rag.add_documents_list(exchanges)

# Prepare a the grounding tools for use.
tool_wiki = WikiGroundingGenerator(api.args.CLIENT, tool_rag)
tool_ground = SearchGroundingGenerator(api.args.CLIENT, tool_rag)
tool_rest = RestGroundingGenerator(tool_rag, with_limits=True)

Generate document embedding: 0it [00:00, ?it/s]


## Function Calling Expert

In [21]:
# Implement the callable functions and function handler.

def ask_rag_tool(content):
    return tool_rag.generate_answer(content["question"]).text

def ask_wiki_tool(content):
    return tool_wiki.generate_answer(content["q"], content["id"])

def ask_search_tool(content):
    return tool_ground.generate_answer(content["q"], content["id"])

def get_exchange_codes_1(content):
    return tool_rag.get_exchange_codes()

def get_exchange_code_1(content):
    return tool_rag.get_exchange_codes(with_query=content)
    
def last_market_close(content):
    return tool_rag.last_market_close(content["exchange"])
    
def get_symbol_1(content, by_name: bool = True):
    stored = tool_rag.get_api_documents(content["query"], content["q"], "get_symbol_1")
    if len(stored) == 0:
        return tool_rest.get_symbol(content, by_name)
    return json.loads(stored[0].docs)

def get_symbols_1(content):
    return None # todo

def get_name_1(content):
    return get_symbol_1(content, by_name = False)

def get_quote_1(content):
    stored = tool_rag.get_api_documents(content["query"], content["symbol"], "get_quote_1")
    if tool_rag.generated_events(content["exchange"]).is_open():
        return get_current_price_1(content)
    elif len(stored) > 0:
        last_close = parse(tool_rag.last_market_close(content["exchange"])).timestamp()
        for quote in stored:
            if quote.meta["timestamp"] >= last_close:
                return [quote.docs for quote in stored]
    return get_current_price_1(content)

def get_current_price_1(content):
    return tool_rest.get_current_price(content)

def get_market_status_1(content):
    stored, has_update = tool_rag.get_market_status(content['exchange'])
    if has_update:
        with_id = stored[0].store_id if len(stored) > 0 else None
        return tool_rest.get_market_status(content, with_id)
    return stored[0].docs

def get_session_1(content):
    return json.loads(get_market_status_1(content))["session"]

def get_peers_1(content):
    stored = tool_rag.get_peers_document(content["query"], content["symbol"], content['grouping'])
    if len(stored) == 0:
        peers = tool_rest.get_peers(content)
        if peers.count > 0:
            names = []
            for peer in peers.get():
                if peer == content["symbol"]:
                    continue # skip including the query symbol in peers
                name = get_name_1(dict(q=peer, exchange=content["exchange"], query=content["query"]))
                if name != Api.Const.Stop():
                    data = {"symbol": peer, "name": name}
                    names.append(data)
            tool_rag.add_peers_document(content["query"], names, content["symbol"], "get_peers_1", content['grouping'])
            return names
        return Api.Const.Stop()
    return json.loads(stored[0].docs)["peers"]

def local_datetime(content):
    local_t = []
    for timestamp in content["t"]:
        local_t.append(local_date_from_epoch(timestamp))
    return local_t

def local_date_from_epoch(timestamp):
    if len(str(timestamp)) == 13:
        return datetime.fromtimestamp(timestamp/1000, tz=GeneratedEvent.tz()).strftime('%c')
    else:
        return datetime.fromtimestamp(timestamp, tz=GeneratedEvent.tz()).strftime('%c')

def get_financials_1(content):
    stored = tool_rag.get_basic_financials(content["query"], content["symbol"], "get_financials_1")
    if len(stored) == 0:
        return tool_rest.get_basic_financials(content)
    return [chunk.docs for chunk in stored]

def get_news_1(content):
    stored = tool_rag.get_api_documents(content["query"], content["symbol"], "get_news_1")
    if len(stored) == 0:
        return tool_rest.get_news_simple(content)
    return [NewsTypeFinn.model_validate_json(news.docs).summary().model_dump_json() for news in stored]

def get_daily_candle_2(content):
    stored = tool_rag.get_api_documents(
        query=content["query"], topic=content["stocksTicker"], source="daily_candle_2", 
        meta_opt=[{"from_date": content["date"], "adjusted": content["adjusted"]}])
    if len(stored) == 0:
        candle = tool_rest.get_daily_candle(content)
        # Attempt to recover from choosing a holiday.
        candle_date = parse(content["date"])
        if candle.status is RestStatus.NONE and candle_date.weekday() == 0 or candle_date.weekday() == 4:
            if candle_date.weekday() == 0: # index 0 is monday, index 4 is friday
                content["date"] = candle_date.replace(day=candle_date.day-3).strftime("%Y-%m-%d")
            else:
                content["date"] = candle_date.replace(day=candle_date.day-1).strftime("%Y-%m-%d")
            return get_daily_candle_2(content)
        return candle.model_dump_json()
    return [json.loads(candle.docs) for candle in stored]

def get_custom_candle_2(content):
    stored = tool_rag.get_api_documents(
        query=content["query"], topic=content["stocksTicker"], source="custom_candle_2", 
        meta_opt=[{
            "timespan": content["timespan"],
            "adjusted": content["adjusted"],
            "from": content["from"],
            "to": content["to"]}])
    if len(stored) == 0:
        return tool_rest.get_custom_candle(content)
    return [json.loads(candle.docs) for candle in stored]

def get_overview_2(content):
    stored = tool_rag.get_api_documents(content["query"], content["ticker"], "ticker_overview_2")
    if len(stored) == 0:
        return tool_rest.get_overview(content)
    return json.loads(stored[0].docs)

def get_trends_1(content):
    stored = tool_rag.get_api_documents(content["query"], content["symbol"], "trends_1")
    if len(stored) == 0:
        return tool_rest.get_trends_simple(content)
    return [json.loads(trend.docs) for trend in stored]

def get_news_2(content):
    timestamp_from = parse(content["published_utc.gte"]).timestamp()
    timestamp_to = parse(content["published_utc.lte"]).timestamp()
    news_from = tool_rag.get_api_documents(
        content["query"], content["ticker"], "get_news_2", [{"published_utc": timestamp_from}])
    news_to = tool_rag.get_api_documents(
        content["query"], content["ticker"], "get_news_2", [{"published_utc": timestamp_to}])
    if len(news_from) > 0 and len(news_to) > 0:
        stored = tool_rag.get_api_documents(
            content["query"], content["ticker"], "get_news_2",
            [{"published_utc": {"$gte": timestamp_from}},
             {"published_utc": {"$lte": timestamp_to}}])
        return [NewsTypePoly.model_validate_json(news.docs).summary().model_dump_json() for news in stored]
    return tool_rest.get_news_tagged(content)
        
finance_tool = types.Tool(
    function_declarations=[
        decl_get_symbol_1,
        decl_get_symbols_1,
        decl_get_name_1,
        decl_get_symbol_quote_1,
        decl_get_market_status_1,
        decl_get_market_session_1,
        decl_get_company_peers_1,
        decl_get_local_datetime,
        decl_get_last_market_close,
        decl_get_exchange_codes_1,
        decl_get_exchange_code_1,
        decl_get_financials_1,
        decl_get_daily_candlestick_2,
        decl_get_custom_candlestick_2,
        decl_get_ticker_overview_2,
        decl_get_recommendation_trends_1,
        decl_get_news_with_sentiment_2,
        decl_get_rag_tool_response,
        decl_get_wiki_tool_response,
        decl_get_search_tool_response
    ]
)

function_handler = {
    "get_symbol_1": get_symbol_1,
    "get_symbols_1": get_symbols_1,
    "get_name_1": get_name_1,
    "get_symbol_quote_1": get_quote_1,
    "get_market_status_1": get_market_status_1,
    "get_market_session_1": get_session_1,
    "get_company_peers_1": get_peers_1,
    "get_local_datetime": local_datetime,
    "get_last_market_close": last_market_close,
    "get_exchange_codes_1": get_exchange_codes_1,
    "get_exchange_code_1": get_exchange_code_1,
    "get_financials_1": get_financials_1,
    "get_daily_candlestick_2": get_daily_candle_2,
    "get_custom_candlestick_2": get_custom_candle_2,
    "get_ticker_overview_2": get_overview_2,
    "get_recommendation_trends_1": get_trends_1,
    "get_news_with_sentiment_2": get_news_2,
    "get_rag_tool_response": ask_rag_tool,
    "get_wiki_tool_response": ask_wiki_tool,
    "get_search_tool_response": ask_search_tool
}

In [22]:
# Implement the function calling expert.
# Define the system prompt.
instruction = f"""You are a helpful and informative bot that answers finance and stock market questions. 
Only answer the question asked and do not change topic. While the answer is still
unknown you must follow these rules for predicting function call order:

RULE#1: Always consult your other functions before get_search_tool_response.
RULE#2: Always consult get_wiki_tool_response before get_search_tool_response.
RULE#3: Always consult get_search_tool_response last.
RULE#4: Always convert timestamps with get_local_datetime and use the converted date/time in your response.
RULE#5: Always incorporate as much useful information from tools and functions in your response."""

def get_response():
    # Enable system prompt, function calling and minimum-randomness.
    config_fncall = types.GenerateContentConfig(
        system_instruction=instruction,
        tools=[finance_tool],
        temperature=0.0
    )
    memory.response = api.retriable(
        api.args.CLIENT.models.generate_content,
        model=api(Api.Model.GEN),
        config=config_fncall,
        contents=memory.contents)

def retry_last_send():
    api.generation_fail()
    time.sleep(api.dt_between)
    get_response()

@retry.Retry(
    predicate=is_retriable,
    initial=2.0,
    maximum=64.0,
    multiplier=2.0,
    timeout=600,
)
def send_message(prompt):
    #display(Markdown("#### Prompt"))
    #print(prompt, "\n")
    memory.set_prompt(prompt)
    # Handle cases with multiple chained function calls.
    function_calling_in_process = True
    # Send the initial user prompt and function declarations.
    get_response()
    while function_calling_in_process:
        try:
            response_parts = memory.response.candidates[0].content.parts
            # A summary response never includes function calls.
            if not any(part.function_call for part in response_parts):
                memory.set_summary("\n".join(e.text for e in response_parts))
                function_calling_in_process = False
                break # The function calling chain is complete.
            else:
                # A part can be a function call or reasoning-step.
                for part in response_parts:
                    if function_call := part.function_call:
                        # Extract the function call.
                        fn_name = function_call.name
                        #display(Markdown("#### Predicted function name"))
                        #print(fn_name, "\n")
                        # Extract the function call arguments.
                        fn_args = {key: value for key, value in function_call.args.items()}
                        #display(Markdown("#### Predicted function arguments"))
                        #print(fn_args, "\n")
                        # Call the predicted function.
                        print("send_message: get function response")
                        api_response = function_handler[fn_name](fn_args)[:20000] # Stay within the input token limit
                        #display(Markdown("#### API response"))
                        #print(api_response[:500], "...", "\n")
                        # Create an API response part.
                        api_response_part = types.Part.from_function_response(
                            name=fn_name,
                            response={"content": api_response},
                        )
                        memory.update_contents(function_call, api_response_part)
                    else:
                        #display(Markdown("#### Natural language reasoning step"))
                        #print(part.text)
                        memory.set_reason(part.text)
                print("send_message: updating state")
                get_response() # Send the updated prompt.
                print("send_message: got a response")
        except Exception as e:
            if isinstance(response_parts, list):
                print("send_message: generated wrong function arguments")
            retry_last_send()
            
    # Show the final natural language summary.
    display(Markdown("#### Natural language response"))
    display(Markdown(memory.summary))

# RAG Baseline Check

In [23]:
response = tool_rag.get_exchanges_csv(
    """Give me a dictionary in string form. It must contain key:value pairs mapping 
    exchange code to name. Just the dictionary string in pretty form.""")
print(response.candidates[0].content.parts[-1].text)

response = tool_rag.get_exchanges_csv(
    """What is the Germany exchange code? Return only the exchange codes as a simple 
    comma separated value that I can copy.""")
print(response.candidates[0].content.parts[-1].text, "\n")

response = tool_rag.get_exchanges_csv("What are the Germany exchanges and thier corresponding exchange codes?")
print(response.text, "\n")

response = tool_rag.generate_answer("What are Google's stock ticker symbols?")
print(response.text, "\n")

response = tool_rag.generate_answer("What is Facebook's stock ticker symbol?")
print(response.text, "\n")

response = tool_rag.get_exchanges_csv("What are the US exchange operating hours?")
print(response.text, "\n")

response = tool_rag.get_exchanges_csv(
    f"""Answer based on your knowledge of exchange operating hours.
    Do not answer in full sentences. Omit all chat and provide the answer only.
    The fields pre_market and post_market both represent extended operating hours.

    The current date and time: {datetime.now(GeneratedEvent.tz()).strftime('%c')}

    Weekdays are: Mon, Tue, Wed, Thu, Fri.
    On weekdays all exchanges open after pre-market and regular hours.
    On weekdays all exchanges close after regular and post-market hours.
    
    Weekends are: Sat, Sun.
    Always exclude weekends from exchange operating hours.
    A list of holidays in date format mm-dd-yyyy: {tool_rag.holidays["US"]}
    Always exclude holidays from exchange operating hours.
    When the answer is a holiday use the prior weekday for close.
    When the answer is a holiday use the next weekday for open.
    
    Consider the US exchange's operating hours.
    Provide the most recent weekday's close including post_market hours.
    
    Answer with a date that uses this format: '%a %b %d %X %Y'.""")
print(response.candidates[0].content.parts[-1].text)

```
{
    "VN": "Vietnam exchanges including HOSE, HNX and UPCOM",
    "AD": "ABU DHABI SECURITIES EXCHANGE",
    "US": "US exchanges (NYSE, Nasdaq)",
    "CO": "OMX NORDIC EXCHANGE COPENHAGEN A/S",
    "QA": "QATAR EXCHANGE",
    "BA": "BOLSA DE COMERCIO DE BUENOS AIRES",
    "MX": "BOLSA MEXICANA DE VALORES (MEXICAN STOCK EXCHANGE)",
    "PR": "PRAGUE STOCK EXCHANGE",
    "HK": "HONG KONG EXCHANGES AND CLEARING LTD",
    "CA": "Egyptian Stock Exchange",
    "AX": "ASX - ALL MARKETS",
    "SX": "DEUTSCHE BOERSE Stoxx",
    "KQ": "KOREA EXCHANGE (KOSDAQ)",
    "DB": "DUBAI FINANCIAL MARKET",
    "PM": "Philippine Stock Exchange",
    "KS": "KOREA EXCHANGE (STOCK MARKET)",
    "ST": "NASDAQ OMX NORDIC STOCKHOLM",
    "DU": "BOERSE DUESSELDORF",
    "TL": "NASDAQ OMX TALLINN",
    "AT": "ATHENS EXCHANGE S.A. CASH MARKET",
    "SW": "SWISS EXCHANGE",
    "LS": "NYSE EURONEXT - EURONEXT LISBON",
    "SI": "SINGAPORE EXCHANGE",
    "RG": "NASDAQ OMX RIGA",
    "CR": "CARACAS STOCK EXCHANGE"

# SC1 Baseline Check

In [24]:
# Wait 59s for rate-limits to reset on FREE-tier.
if api.args.API_LIMIT is Api.Limit.FREE.value:
    print("Gemini API limit is FREE. Waiting 59s...")
    time.sleep(59)

Gemini API limit is FREE. Waiting 59s...
Api.refill_rpm 10


In [25]:
send_message("What is the current session for US exchanges?")

send_message: get function response


Generate US->MarketEvent.POST_CLOSE: 100%|██████████| 1/1 [00:01<00:00,  1.92s/it]
Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

The current market session for US exchanges is post-market.

In [26]:
send_message("What is the US market status?")

Api.generation_fail.next_model: model is now gemini-2.5-flash-preview-09-2025
send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response
send_message: updating state
send_message: got a response


#### Natural language response

The US market is currently **open** and in the **post-market** session as of Mon Nov 24 16:27:21 2025 (America/New_York timezone). It is not a holiday.

In [27]:
send_message("When was the last US market close?")

send_message: get function response
send_message: updating state
send_message: got a response


#### Natural language response

The last US market close was on Friday, November 21, 2025, at 8:00:00 PM.

In [28]:
send_message("What is Apple's stock ticker?")

send_message: get function response


Score similarity to query: 100%|██████████| 10/10 [00:00<00:00, 17.35it/s]
Generate api embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

The stock ticker for Apple is AAPL.

In [29]:
send_message("What is the current price of Amazon stock? Display the result as a json string in markdown.")

send_message: get function response


Score similarity to query: 100%|██████████| 2/2 [00:00<00:00,  7.00it/s]
Generate api embedding: 0it [00:00, ?it/s]


send_message: updating state
Api.generation_fail.next_model: model is now gemini-2.0-flash-exp
send_message: got a response
send_message: get function response


Generate quote embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response
send_message: get function response
send_message: updating state
send_message: got a response


#### Natural language response

The current price of Amazon stock (AMZN) is $226.06 as of Mon Nov 24 16:00:00 2025. The change from the previous close is $5.37, representing a 2.4333% increase. The high for the day is $227.33, and the low is $222.27. The opening price for the day was $222.555, and the previous close price was $220.69.

```json
{
"c": 226.06,
"d": 5.37,
"dp": 2.4333,
"h": 227.33,
"l": 222.27,
"o": 222.555,
"pc": 220.69,
"t": 1764018000
}
```

In [30]:
send_message("""Show me Apple's basic financials and help me understand key performance metrics. 
How has the stock performed?""")

send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

Here's an overview of Apple's financial performance based on the provided data:

**Financial Health & Performance Metrics:**

*   **Profitability:**
    *   **Net Profit Margin (TTM):** 26.92%, indicating strong profitability.
    *   **Operating Margin (TTM):** 31.97%, reflecting efficient operations.
    *   **Gross Margin (TTM):** 46.91%, showcasing a healthy ability to manage production costs.
    *   **Return on Equity (ROE) (TTM):** 164.05%, suggesting effective use of shareholder equity.
    *   **Return on Assets (ROA) (TTM):** 32.80%, indicating good asset utilization.

*   **Revenue & Growth:**
    *   **Revenue Growth (TTM YoY):** 6.43%, demonstrating continued growth.
    *   **EPS Growth (TTM YoY):** 22.89%, indicating strong earnings growth.
    *   **Revenue Per Share (TTM):** 27.9987
    *   **Asset Turnover TTM:** 1.2186, reflecting efficient asset management to generate revenue.

*   **Liquidity & Solvency:**
    *   **Current Ratio (Annual):** 0.8933, suggesting potential short-term liquidity challenges.
    *   **Quick Ratio (Annual):** 0.8588, reinforcing the need to monitor short-term obligations.
    *   **Long Term Debt/Equity (Annual):** 1.0623, indicating a moderate level of debt relative to equity.

*   **Valuation:**
    *   **Price-to-Earnings Ratio (P/E) (TTM):** 36.278, suggesting investors are willing to pay a premium for Apple's earnings.
    *   **Forward P/E:** 33.52, indicating expected future earnings.
    *   **Price-to-Book Ratio (P/B):** 55.111, reflecting a high market valuation compared to book value.
    *   **PEG Ratio (TTM):** 1.59, suggesting the stock might be slightly overvalued relative to its earnings growth.

*   **Stock Performance:**
    *   **52 Week Price Return Daily:** 18.55%
    *   **Year-to-Date Price Return Daily:** 8.41%
    *   **13 Week Price Return Daily:** 19.2%
    *   **5 Day Price Return Daily:** 1.51%

**Key Observations & Potential Interpretations:**

*   Apple exhibits strong profitability, efficient operations, and good asset utilization.
*   The company demonstrates continued revenue and earnings growth.
*   Liquidity ratios suggest a need for careful management of short-term obligations.
*   Valuation metrics indicate that the stock is trading at a premium.
*   Stock performance has been positive over the past year.

**Additional Considerations:**

*   It's important to compare these metrics to Apple's industry peers and historical performance to gain a more comprehensive understanding.
*   Analyzing the trends in these metrics over time can provide valuable insights into the company's long-term performance.
*   External factors, such as economic conditions and industry trends, can also impact Apple's financial performance.


In [31]:
send_message("I need Apple's daily candlestick from 2025-05-05")

send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

OK. On 2025-05-05, Apple's stock (AAPL) had the following daily candlestick data:
*   Open: 203.1
*   High: 204.1
*   Low: 198.21
*   Close: 198.89
*   Volume: 69018452
*   PreMarket: 205.0
*   AfterHours: 198.6

In [32]:
send_message("Tell me who are Apple's peers?")

send_message: get function response


Score similarity to query: 100%|██████████| 5/5 [00:00<00:00, 32.60it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  7.96it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  7.84it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 5/5 [00:00<00:00, 37.55it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  5.94it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  7.22it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 2/2 [00:00<00:00, 14.26it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  6.07it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 2/2 [00:00<00:00, 13.04it/s]

send_message: updating state
Api.generation_fail.next_model: model is now gemini-2.0-flash
send_message: got a response


#### Natural language response

Here are some of Apple's peers: DELL (DELL TECHNOLOGIES -C), WDC (WESTERN DIGITAL CORP), SNDK (SANDISK CORP), HPE (HEWLETT PACKARD ENTERPRISE), PSTG (PURE STORAGE INC - CLASS A), HPQ (HP INC), NTAP (NETAPP INC), SMCI (SUPER MICRO COMPUTER INC), IONQ (IONQ INC), QUBT (QUANTUM COMPUTING INC), and CMPO (COMPOSECURE INC-A).


In [33]:
send_message("Tell me who are Amazon's peers?")

send_message: get function response


Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  7.64it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 2/2 [00:00<00:00, 16.42it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  7.69it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 5/5 [00:00<00:00, 37.76it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 11/11 [00:00<00:00, 65.73it/s]
Generate api embedding: 0it [00:00, ?it/s]


Api.zero_error: model is now gemini-2.5-flash


Score similarity to query: 100%|██████████| 2/2 [00:00<00:00, 15.11it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  7.22it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  7.39it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  7.07it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  7.97it/s]
Generate api embedding: 0it [00:00, ?it/s]
Generate peers embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

Amazon's peers include Coupang Inc (CPNG), eBay Inc (EBAY), Dillard's Inc-CL A (DDS), Ollie's Bargain Outlet Holdi (OLLI), Macy's Inc (M), Etsy Inc (ETSY), Pattern Group Inc-CL A (PTRN), Kohl's Corp (KSS), Savers Value Village Inc (SVV), and Groupon Inc (GRPN).

In [34]:
send_message("""Locate Apple's stock ticker, then download recommendation trends of all Apple's peers by sub-industry, 
and then finally compare them.""")

send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: get function response


...

send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

Here's a comparison of the latest recommendation trends (as of November 2025) for Apple's peers in the sub-industry:

*   **DELL TECHNOLOGIES -C (DELL):** 18 Buy, 6 Hold, 0 Sell, 8 Strong Buy, 0 Strong Sell
*   **WESTERN DIGITAL CORP (WDC):** 19 Buy, 7 Hold, 0 Sell, 6 Strong Buy, 0 Strong Sell
*   **SANDISK CORP (SNDK):** 10 Buy, 8 Hold, 0 Sell, 7 Strong Buy, 0 Strong Sell
*   **HEWLETT PACKARD ENTERPRISE (HPE):** 8 Buy, 12 Hold, 0 Sell, 6 Strong Buy, 0 Strong Sell
*   **PURE STORAGE INC - CLASS A (PSTG):** 13 Buy, 7 Hold, 1 Sell, 6 Strong Buy, 0 Strong Sell
*   **HP INC (HPQ):** 3 Buy, 16 Hold, 1 Sell, 2 Strong Buy, 0 Strong Sell
*   **NETAPP INC (NTAP):** 9 Buy, 15 Hold, 0 Sell, 3 Strong Buy, 0 Strong Sell
*   **SUPER MICRO COMPUTER INC (SMCI):** 10 Buy, 11 Hold, 3 Sell, 2 Strong Buy, 0 Strong Sell
*   **IONQ INC (IONQ):** 10 Buy, 3 Hold, 0 Sell, 2 Strong Buy, 0 Strong Sell
*   **QUANTUM COMPUTING INC (QUBT):** 5 Buy, 2 Hold, 0 Sell, 2 Strong Buy, 0 Strong Sell
*   **COMPOSECURE INC-A (CMPO):** 8 Buy, 1 Hold, 1 Sell, 2 Strong Buy, 0 Strong Sell

In [35]:
send_message("""Tell me Amazon's current share price and provide candlestick data for the past month. 
Sort the data in descending order by date. Format the prices consistently as currency. 
Round prices to two decimal places. 
Present the data with multiple columns for display in markdown. 
Discuss and provide details about any patterns you notice in the price data. 
Correlate recent patterns with news over the same date range.""")

send_message: get function response


Generate quote embedding: 0it [00:00, ?it/s]


send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response
send_message: get function response


Add chunks embedding: 0it [00:00, ?it/s]


send_message: get function response
send_message: updating state
Api.generation_fail.next_model: model is now gemini-2.5-flash-preview-09-2025
send_message: got a response
send_message: get function response
send_message: updating state
send_message: got a response


#### Natural language response

The current share price for **Amazon (AMZN)** is **$226.06** as of **Mon Nov 24 16:00:00 2025**. The stock is up **$5.37** (+2.43%) from the previous close.

Here is the daily candlestick data for Amazon's stock price over the past month, sorted in descending order by date:

| Date | Open | High | Low | Close | Volume |
| :--- | :--- | :--- | :--- | :--- | :--- |
| Fri Nov 21 00:00:00 2025 | $216.35 | $222.21 | $215.18 | $220.69 | 68,490,453 |
| Thu Nov 20 00:00:00 2025 | $227.05 | $227.41 | $216.74 | $217.14 | 50,308,862 |
| Wed Nov 19 00:00:00 2025 | $223.74 | $223.74 | $218.52 | $222.69 | 58,335,353 |
| Tue Nov 18 00:00:00 2025 | $228.10 | $230.20 | $222.42 | $222.55 | 60,608,442 |
| Mon Nov 17 00:00:00 2025 | $233.25 | $234.60 | $229.19 | $232.87 | 59,918,908 |
| Fri Nov 14 00:00:00 2025 | $235.06 | $238.73 | $232.89 | $234.69 | 38,956,619 |
| Thu Nov 13 00:00:00 2025 | $243.05 | $243.75 | $236.50 | $237.58 | 41,401,638 |
| Wed Nov 12 00:00:00 2025 | $250.24 | $250.37 | $243.75 | $244.20 | 31,190,063 |
| Tue Nov 11 00:00:00 2025 | $248.41 | $249.75 | $247.23 | $249.10 | 23,563,960 |
| Mon Nov 10 00:00:00 2025 | $248.34 | $251.75 | $245.59 | $248.40 | 36,476,474 |
| Fri Nov 7 00:00:00 2025 | $242.90 | $244.90 | $238.49 | $244.41 | 46,374,294 |
| Thu Nov 6 00:00:00 2025 | $249.16 | $250.38 | $242.17 | $243.04 | 46,004,201 |
| Wed Nov 5 00:00:00 2025 | $249.03 | $251.00 | $246.16 | $250.20 | 40,610,602 |
| Tue Nov 4 00:00:00 2025 | $250.38 | $257.01 | $248.66 | $249.32 | 51,546,311 |
| Mon Nov 3 00:00:00 2025 | $255.36 | $258.60 | $252.90 | $254.00 | 95,997,714 |
| Fri Oct 31 00:00:00 2025 | $250.10 | $250.50 | $243.98 | $244.22 | 166,340,683 |
| Thu Oct 30 00:00:00 2025 | $227.06 | $228.44 | $222.75 | $222.86 | 102,252,888 |
| Wed Oct 29 00:00:00 2025 | $231.67 | $232.82 | $227.76 | $230.30 | 52,035,936 |
| Tue Oct 28 00:00:00 2025 | $228.22 | $231.49 | $226.21 | $229.25 | 47,099,924 |
| Mon Oct 27 00:00:00 2025 | $227.66 | $228.40 | $225.54 | $226.97 | 38,266,995 |
| Fri Oct 24 00:00:00 2025 | $221.97 | $225.40 | $221.90 | $224.21 | 38,684,853 |

***

### Price Pattern Analysis and Correlation with News

The price data for Amazon (AMZN) over the past month (October 24, 2025, to November 21, 2025) reveals a distinct **V-shaped pattern** characterized by a sharp surge followed by a gradual decline and a recent stabilization.

#### 1. The Initial Surge (Late October to Early November)
*   **Pattern:** The stock experienced a significant and rapid upward movement, starting from a low of **$224.21** on October 24 and peaking at **$254.00** on November 3. This represents a gain of approximately **13.37%** in just over a week. The volume on October 31 (**166.34 million**) and November 3 (**95.99 million**) was exceptionally high, confirming strong buying pressure.
*   **Correlation with News:** This surge is directly correlated with a series of highly positive news events, primarily centered around Amazon's cloud computing division, **AWS**, and its strategic positioning in the **Artificial Intelligence (AI)** race:
    *   **October 31:** Amazon reported **strong Q3 earnings** with AWS revenue growing 20% year-over-year, causing the stock to surge 10% in a single day (Nasdaq 100 Rebounds, Amazon Jumps 10% On Strong Earnings).
    *   **November 3-4:** News broke of a massive **$38 billion cloud infrastructure deal with OpenAI**, which will utilize AWS servers and hundreds of thousands of Nvidia chips. This news was a major catalyst, with the stock jumping 5% on November 3 alone (Amazon Strikes $38B OpenAI Deal, Why Did Amazon Stock Jump 5% Today?).
    *   **General Sentiment:** Multiple articles during this period highlighted Amazon's strong AI potential, diversified business model, and attractive valuation, with analysts raising price targets.

#### 2. The Correction/Pullback (Mid-November)
*   **Pattern:** Following the peak on November 3, the stock entered a clear downtrend, dropping from **$254.00** to a low of **$217.14** on November 20. This represents a decline of approximately **14.51%**. The trading volume remained relatively high during this period, suggesting significant selling pressure.
*   **Correlation with News:** This sharp pullback aligns with broader market skepticism and specific concerns about AI investment sustainability:
    *   **November 13:** Tech stocks experienced a massive market value decline of over $700 billion due to skepticism about Federal Reserve rate cuts and potential AI infrastructure bottlenecks, with Amazon shedding around **$70 billion in market value** (Tech Stocks Wipe Out Over $700 Billion As Traders Flee AI Hype).
    *   **November 18:** A major system failure at Cloudflare caused a global web services outage, impacting AWS services and contributing to a broader tech stock selloff, with AMZN's stock price declining by 3.29% (Cloudflare Stumbled — And Tech Stocks Extend The Selloff).
    *   **November 19-21:** Concerns about AI overspending and high valuations continued to pressure the stock. An article on November 21 noted a **12% stock decline** due to concerns about an AWS cloud optimization slowdown and potential impact of reduced consumer discretionary spending (3 Big Tech Stocks Sliding: What’s Behind the Drop?).

#### 3. Recent Stabilization (Late November)
*   **Pattern:** In the final days of the data (November 19-21), the stock appears to be attempting to stabilize, with the closing price on November 21 at **$220.69**, and the current price on November 24 at **$226.06**.
*   **Correlation with News:** The recent stabilization and slight rebound on November 24 is supported by renewed positive sentiment:
    *   **November 24:** News reported that Amazon gained nearly 2% after announcing a **$50 billion AI and supercomputing infrastructure investment**, suggesting the company is doubling down on its AI strategy despite earlier market jitters (Stocks Soar, Nasdaq 100 Eyes Best Day In 6 Months).
    *   **General Sentiment:** Positive analyst commentary continued to emerge, highlighting Amazon's strong growth in AWS, advertising, and AI-powered chatbot Rufus, reinforcing its long-term growth potential (My 2 Favorite Stocks to Buy Right Now).

In summary, Amazon's stock price over the past month was highly volatile, driven almost entirely by the market's reaction to its **AI and cloud computing strategy**. The initial surge was a direct response to strong earnings and the massive OpenAI deal, while the subsequent sharp decline reflected broader market concerns about AI investment costs and a temporary service outage. The stock is currently attempting to recover, supported by the company's continued commitment to massive AI infrastructure spending.

In [36]:
send_message("What is Apple's ticker overview")

send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

Apple Inc. (AAPL) is a major global company known for its hardware and software products, including the iPhone, Mac, iPad, and Watch.

Here is an overview of the company:

*   **Name:** Apple Inc.
*   **Ticker:** AAPL
*   **Primary Exchange:** XNAS (NASDAQ)
*   **Market Cap:** $4,011,632,075,970.00
*   **CIK:** 0000320193
*   **Phone Number:** (408) 996-1010
*   **Address:** ONE APPLE PARK WAY, CUPERTINO, CA, 95014
*   **Total Employees:** 166,000
*   **List Date:** 1980-12-12
*   **SIC Description:** ELECTRONIC COMPUTERS
*   **Homepage:** https://www.apple.com

**Description:**
Apple is one of the largest companies in the world. Its portfolio is centered around the iPhone, which accounts for a majority of its sales. Products like the Mac, iPad, and Watch are designed to integrate within the expansive Apple software ecosystem. The company is also expanding into new applications such as streaming video, subscription bundles, and augmented reality. Apple designs its own software and semiconductors, with manufacturing handled by subcontractors like Foxconn and TSMC. Sales are made both directly through its flagship stores and indirectly through partnerships and distribution channels.

In [37]:
send_message("What is Google's stock ticker symbol?")

send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response


Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Generate wiki embeddings: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

Google's stock is listed on the NASDAQ stock exchange under two ticker symbols: **GOOGL** and **GOOG**.

These symbols now represent **Alphabet Inc.**, which is Google's holding company, a change that took effect in the fourth quarter of 2015. The company is also listed on the Frankfurt Stock Exchange under the ticker symbol **GGQ1**.

In [38]:
send_message("What is MGM Studio's stock symbol?")

Api.generation_fail.next_model: model is now gemini-2.0-flash-exp
send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response


Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Generate wiki embeddings: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response
send_message: get function response
send_message: updating state
send_message: got a response


#### Natural language response

I am unable to find the stock symbol for MGM Studios.


In [39]:
send_message("What is MGM Studio's owner company stock symbol?")

send_message: get function response
send_message: updating state
Api.generation_fail.next_model: model is now gemini-2.0-flash
Api.zero_error: model is now gemini-2.5-flash
send_message: got a response
send_message: get function response
send_message: updating state
send_message: got a response


#### Natural language response

MGM Studios is owned by Amazon. The stock symbol for Amazon is AMZN.

In [40]:
send_message("What is Facebook's stock ticker symbol?")

send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response


Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Generate wiki embeddings: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

The stock ticker symbol for Facebook, which is now Meta Platforms, Inc., is META. The company trades on the Nasdaq exchange.

In [41]:
send_message("""Compare Amazon's bullish versus bearish predictions from Oct 01 2025 until today. 
Include a discussion of recommendation trends, and sentiment analysis of news from the same dates. 
Discuss any patterns or correlations you find.""")

send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: get function response
send_message: generated wrong function arguments
Api.generation_fail.next_model: model is now gemini-2.5-flash-preview-09-2025
send_message: get function response


Add chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
Api.zero_error: model is now gemini-2.5-flash
send_message: got a response


#### Natural language response

Based on the analysis of analyst recommendations and news sentiment for Amazon (AMZN) from October 1, 2025, until today, November 24, 2025, the predictions are overwhelmingly **bullish**, driven by the company's strategic pivot toward Artificial Intelligence (AI) and the continued strength of its cloud computing division.

### **Recommendation Trends (Bullish vs. Bearish)**

Analyst sentiment for Amazon during this period was near-unanimously bullish:

| Metric | Oct 1, 2025 | Nov 1, 2025 | Trend |
| :--- | :--- | :--- | :--- |
| **Bullish (Strong Buy + Buy)** | 75 | 76 | Slight Increase |
| **Neutral (Hold)** | 3 | 2 | Slight Decrease |
| **Bearish (Sell + Strong Sell)** | 0 | 0 | None |

**Conclusion:** The analyst community maintained a **Strong Buy** consensus throughout the period, with zero "Sell" or "Strong Sell" recommendations. This indicates a high degree of confidence in Amazon's long-term growth trajectory.

***

### **Sentiment Analysis of News (Oct 1, 2025 - Nov 24, 2025)**

The news coverage was predominantly positive, with bullish articles outnumbering bearish ones by a ratio of over 5-to-1.

#### **Bullish Predictions and Themes**

The core of the bullish case is centered on Amazon's high-margin, high-growth segments:

*   **AWS Dominance and Acceleration:** News repeatedly highlighted the accelerating growth of Amazon Web Services (AWS), which reported a **20% year-over-year revenue increase** in its Q3 earnings. AWS is consistently cited as the company's most lucrative segment, generating a significant portion of its operating profit.
*   **Strategic AI Investments:** The market reacted strongly to Amazon's aggressive AI strategy, including:
    *   A massive **$38 billion cloud infrastructure deal with OpenAI** (reported early November).
    *   Commitments to **$50 billion in AI and supercomputing infrastructure** investment.
    *   Development of **custom AI chips** (Trainium and Inferentia) and AI-powered tools like the Rufus chatbot.
*   **Strong Financial Performance:** The Q3 earnings report (late October/early November) was a major catalyst, with the stock surging up to 10% after reporting **13% sales growth** and a **38% increase in net income**. Analysts subsequently raised price targets, projecting up to **50% potential stock price growth** over the next three years.
*   **E-commerce and Advertising Strength:** Positive sentiment was also tied to the continued dominance of the e-commerce platform (40% U.S. market share) and the rapid growth of the high-margin digital advertising business (up 22% YoY).

#### **Bearish Predictions and Themes**

The bearish and mixed sentiment focused on short-term risks and the high cost of the AI pivot:

*   **High Capital Expenditure Risk:** Several articles expressed concern that Amazon's massive AI infrastructure spending (projected to be over $100 billion) could lead to a **45% decline in Free Cash Flow (FCF)** in 2025, creating uncertainty about the return on investment.
*   **Massive Job Cuts:** The company announced plans to cut up to **30,000 corporate jobs** (nearly 10% of its corporate workforce) across multiple divisions, including HR. While this is a cost-cutting measure aimed at efficiency, it was widely reported as a negative for the labor market and a sign of aggressive restructuring.
*   **Operational Setbacks:** The company faced negative press and stock drops following a **major AWS service outage** in late October, which temporarily impacted thousands of websites and services.

***

### **Patterns and Correlations**

A clear pattern emerges from the data:

**The market is willing to overlook short-term operational and financial risks for long-term AI growth.**

*   **Correlation:** The near-unanimous **bullish analyst consensus** is directly correlated with the **strategic AI and AWS growth narrative** in the news. Analysts view the massive AI spending and the resulting FCF pressure as a necessary, high-stakes investment that will cement Amazon's dominance in the next decade.
*   **Pattern:** The negative news events (job cuts, FCF concerns, AWS outage) caused temporary stock price drops and generated bearish commentary, but these were quickly overshadowed by the positive Q3 earnings and the announcement of major AI deals (like the OpenAI partnership). This suggests that investors are prioritizing Amazon's long-term potential as an **AI infrastructure and cloud leader** over its short-term operational challenges. The market is essentially betting that the aggressive cost-cutting (layoffs) will fund the massive AI investment, leading to superior profitability in the future.

In [42]:
send_message("""Compare Google's bullish versus bearish predictions from Oct 01 2025 until today. 
Include a discussion of recommendation trends, and sentiment analysis of news from the same dates. 
Discuss any patterns or correlations you find.""")

send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response


Generate grounding embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response
send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response
send_message: get function response


Add chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
Api.generation_fail.next_model: model is now gemini-2.5-flash-preview-09-2025
send_message: got a response


#### Natural language response

Google's parent company, Alphabet Inc., trades under the ticker symbols **GOOGL** (Class A) and **GOOG** (Class C). The predictions for the company from October 1, 2025, to November 24, 2025, were overwhelmingly bullish across both formal analyst recommendations and real-time news sentiment.

Here is a detailed comparison and analysis of the trends and sentiment during this period.

---

### 1. Recommendation Trends (Bullish vs. Bearish Predictions)

Analyst sentiment for Alphabet was unanimously bullish throughout October and November 2025, with **zero** bearish ratings.

| Period | Strong Buy | Buy | Hold | Sell | Strong Sell | Total Bullish | Total Bearish |
| :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- |
| **2025-10-01** | 21 | 39 | 13 | 0 | 0 | **60** | **0** |
| **2025-11-01** | 21 | 41 | 12 | 0 | 0 | **62** | **0** |

*   **Bullish Prediction:** The consensus was overwhelmingly positive, with 60 "Strong Buy" or "Buy" ratings in October, which slightly increased to 62 by November. This indicates a sustained and growing confidence among analysts in the company's future performance.
*   **Bearish Prediction:** There were no "Sell" or "Strong Sell" recommendations, suggesting analysts saw no fundamental reason for a significant decline in the stock price. The few "Hold" ratings (13 in October, 12 in November) typically reflect a belief that the stock is fairly valued at its current price, rather than a bearish outlook.

### 2. Sentiment Analysis of News (Bullish vs. Bearish Predictions)

An analysis of news articles from October 1, 2025, to November 24, 2025, reveals a heavily positive sentiment, driven by major corporate and market developments.

#### Bullish Themes (Positive Sentiment)

The vast majority of news articles focused on positive developments, with approximately **70 distinct positive mentions** for GOOGL/GOOG. Key bullish themes included:

*   **Record Financial Performance:** The company reported its first-ever **$100 billion quarterly revenue** in Q3, with strong double-digit growth in Google Services and a **34% year-over-year surge in Google Cloud** revenue.
*   **AI Leadership and Monetization:** News highlighted the successful integration of the **Gemini AI model** across its ecosystem, driving growth in both Search and Cloud. Analysts praised Alphabet's AI strategy as "healthy" and "disciplined," contrasting it favorably with competitors.
*   **Institutional Confidence:** A massive vote of confidence came from **Warren Buffett's Berkshire Hathaway**, which disclosed a significant **$4.3 billion to $5 billion stake** in Alphabet in Q3, a move widely interpreted as a strong endorsement of the company's value and AI potential.
*   **Strategic Wins:** The company secured a multi-million dollar cloud infrastructure contract with **NATO** and announced a **$40 billion investment in Texas** for cloud and AI infrastructure, reinforcing its long-term growth commitment.
*   **Valuation:** Alphabet was frequently cited as the **"cheapest"** or **"least overvalued"** among the "Magnificent Seven" tech stocks, making it an attractive buy for value-oriented investors like Buffett.

#### Bearish/Neutral Themes (Negative Sentiment)

Negative sentiment was minimal, with only **4 distinct negative mentions** for GOOGL/GOOG, primarily related to external competitive threats:

*   **Competitive Threats:** The stock experienced a brief drop after **OpenAI launched its ChatGPT Atlas browser**, which was seen as a direct challenge to Google's core search and Chrome dominance.
*   **Legal/Regulatory Risk:** The company was named in a **NYC lawsuit** against social media companies over alleged child addiction, introducing a non-core business risk.
*   **AI Spending Scrutiny:** While generally praised, Alphabet was included in broader market discussions warning that major tech companies might be **"overspending"** on AI infrastructure, raising concerns about the return on investment.

### 3. Patterns and Correlations

There is a **strong, direct correlation** between the formal analyst recommendations and the real-time news sentiment:

1.  **News Validates Recommendations:** The overwhelmingly bullish analyst consensus (zero sells) was consistently validated by the positive news flow. The major events of the period—the record earnings and the Berkshire Hathaway investment—provided concrete, fundamental reasons for analysts to maintain or upgrade their "Buy" and "Strong Buy" ratings.
2.  **AI as the Central Catalyst:** Both sets of data point to **Artificial Intelligence** as the primary driver of bullish sentiment. Analysts are bullish because of Alphabet's AI strategy, and the news is positive because of successful AI execution (Gemini, Cloud growth, custom chips).
3.  **Resilience to Bearish News:** The few negative news items (e.g., the OpenAI browser launch) caused only minor, temporary stock fluctuations, failing to shift the overall market or analyst sentiment. This resilience suggests that the market views Alphabet's core business and AI momentum as strong enough to withstand competitive challenges.

In summary, the period from October 1, 2025, to November 24, 2025, was characterized by a powerful **bullish feedback loop** for Alphabet, where strong financial results and strategic AI execution reinforced the already positive analyst outlook.

In [43]:
send_message("""How is the outlook for Apple based on trends and news sentiment from July 01 2025 until today? 
Perform the same analysis on all peers by sub-industry. Then compare Apple result to it's peers.""")

send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: get function response


Add chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response
send_message: get function response
send_message: updating state
Api.zero_error: model is now gemini-2.5-flash
send_message: got a response
send_message: get function response


...

send_message: updating state
Api.refill_rpm 10
send_message: got a response
Api.generation_fail.next_model: model is now gemini-2.5-flash-preview-09-2025
Api.zero_error: model is now gemini-2.5-flash
send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response


Add chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

Here's an outlook for Apple based on recommendation trends and news sentiment from July 01, 2025, until today (November 24, 2025), along with a comparison to its peers in the Computer Hardware, Storage & Peripherals sub-industry:

**Apple (AAPL) Outlook:**

*   **Recommendation Trends (as of November 1, 2025):** Analysts generally hold a positive view, with 15 "Strong Buy," 23 "Buy," 17 "Hold," and 2 "Sell" recommendations. This indicates a predominantly bullish sentiment from financial experts.
*   **News Sentiment (July 01, 2025 - November 24, 2025):**
    *   **Positive:** News highlights strong iPhone 17 sales, consistent growth in its Services segment, strategic investments in U.S. manufacturing, FDA approval for new Apple Watch health features, and its continued status as a significant holding in many investment funds. Some articles predict Apple's stock could double by 2030, driven by innovation and its ecosystem.
    *   **Negative:** Recurring concerns about Apple lagging in AI innovation compared to competitors, potential impacts from antitrust lawsuits (in China and the EU, and from Elon Musk's X Corp), tariff risks, and reduced production orders for the iPhone Air. Warren Buffett's reduction in his Apple stake is also noted as a cautionary signal.
    *   **Neutral/Mixed:** There's a mixed perception regarding Apple's AI strategy, with some viewing its cautious approach as prudent while others see it as falling behind. Market concentration risks within ETFs are also mentioned.

**Summary for Apple:** Apple's outlook is generally positive from analysts, supported by strong product sales and a growing services division. However, the company faces significant challenges in accelerating its AI development to keep pace with rivals and navigating increasing regulatory scrutiny and trade tensions.

---

**Peer Comparison (Computer Hardware, Storage & Peripherals):**

Apple's peers in this sub-industry exhibit a range of outlooks, largely driven by their direct involvement in the burgeoning AI infrastructure market.

**1. DELL TECHNOLOGIES (DELL):**
*   **Recommendation Trends (as of November 1, 2025):** Very positive, with 8 "Strong Buy" and 18 "Buy" recommendations.
*   **News Sentiment:** Highly positive. Dell is seen as a leader in AI infrastructure, with strong growth in AI server sales (projected over $20 billion), significant contracts, raised financial guidance, and a commitment to increasing dividends. A recent downgrade by Morgan Stanley due to rising memory costs is a minor negative.

**2. WESTERN DIGITAL CORP (WDC):**
*   **Recommendation Trends (as of November 1, 2025):** Very positive, with 6 "Strong Buy" and 19 "Buy" recommendations.
*   **News Sentiment:** Extremely positive. Western Digital is a top S&P 500 performer, driven by high demand for data storage from AI companies. It reported strong earnings and a significant dividend increase. Technical indicators suggest it might be "overbought," indicating a potential short-term pullback.

**3. SANDISK CORP (SNDK):**
*   **Recommendation Trends (as of November 1, 2025):** Positive, with 7 "Strong Buy" and 10 "Buy" recommendations.
*   **News Sentiment:** Strong positive. SanDisk has seen massive stock gains (over 300% this year) due to improving NAND flash market conditions and AI-related storage demand. However, some analysts predict falling memory chip prices in late 2025, and it's also technically "overbought."

**4. HEWLETT PACKARD ENTERPRISE (HPE):**
*   **Recommendation Trends (as of November 1, 2025):** Moderately positive, with 6 "Strong Buy" and 8 "Buy" recommendations.
*   **News Sentiment:** Moderately positive. HPE is strategically expanding into 5G and AI networking through acquisitions and partnerships. While it reported record revenue from AI demand, restructuring costs have impacted profit margins.

**5. PURE STORAGE INC - CLASS A (PSTG):**
*   **Recommendation Trends (as of November 1, 2025):** Positive, with 6 "Strong Buy" and 13 "Buy" recommendations.
*   **News Sentiment:** Positive. Pure Storage is experiencing strong revenue growth, driven by its subscription-based model and key clients in the AI sector.

**6. HP INC (HPQ):**
*   **Recommendation Trends (as of November 1, 2025):** Neutral to slightly positive, with 2 "Strong Buy" and 3 "Buy" recommendations.
*   **News Sentiment:** Neutral to slightly positive. HP is innovating in AI PCs and gaming hardware and focusing on sustainability. However, profitability concerns temper the positive sentiment from its product developments.

**7. NETAPP INC (NTAP):**
*   **Recommendation Trends (as of November 1, 2025):** Moderately positive, with 3 "Strong Buy" and 9 "Buy" recommendations.
*   **News Sentiment:** Moderately positive. NetApp is aligning with new AI-powered storage technologies, as indicated by its collaboration with Broadcom. Limited detailed news sentiment is available.

**8. SUPER MICRO COMPUTER INC (SMCI):**
*   **Recommendation Trends (as of November 1, 2025):** Mixed to slightly positive, with 2 "Strong Buy" and 10 "Buy" recommendations.
*   **News Sentiment:** Mixed. SMCI has strong partnerships with Nvidia and significant AI order backlogs. However, it has faced challenges with missed revenue expectations, declining margins, and negative cash flow, leading to recent stock declines.

**9. IONQ INC (IONQ):**
*   **Recommendation Trends (as of November 1, 2025):** Very positive, with 2 "Strong Buy" and 10 "Buy" recommendations.
*   **News Sentiment:** Highly mixed. IonQ shows promising technological advancements in quantum computing and strategic acquisitions. However, it's characterized by extremely high valuations, minimal revenue, substantial operating losses, and reliance on equity issuances, making it a very high-risk, speculative investment. Recent stock declines are a concern.

**10. QUANTUM COMPUTING INC (QUBT):**
*   **Recommendation Trends (as of November 1, 2025):** Very positive, with 2 "Strong Buy" and 5 "Buy" recommendations.
*   **News Sentiment:** Highly mixed. QUBT has reported strong Q3 earnings and a significant capital raise. However, its extremely high valuation, minimal revenue, and substantial stock declines make it a very high-risk, speculative investment.

**11. COMPOSECURE INC-A (CMPO):**
*   **Recommendation Trends:** No specific recommendation trends available.
*   **News Sentiment:** Limited, but suggests positive strategic moves with a planned business combination.

---

**Comparison of Apple to its Peers:**

*   **AI Leadership:** Apple is generally perceived as *lagging* in AI innovation compared to many of its peers, particularly those directly involved in AI infrastructure like Dell, Western Digital, and SanDisk. These peers are actively reporting significant revenue growth and strategic expansions directly tied to the AI boom. While Apple is investing in AI, its progress is often described as "cautious" or "slow."
*   **Valuation vs. Growth:** Apple's stock is frequently cited as having a high valuation without a corresponding high growth rate, leading to concerns from some investors. In contrast, some AI infrastructure peers (e.g., Dell) are seen as potentially undervalued despite strong AI-driven growth. Quantum computing peers (IonQ, QUBT) have *extremely* high valuations relative to their minimal revenue, highlighting their speculative nature.
*   **Growth Drivers:** Apple's primary growth drivers are iPhone sales and its expanding Services segment. Its peers in the hardware and storage sector are experiencing growth directly from the massive demand for AI servers, data storage, and memory solutions, which is a more direct and immediate benefit from the current AI trend.
*   **Investor Sentiment:** While Apple benefits from a strong brand and loyal customer base, investor sentiment is somewhat cautious due to its AI position and regulatory challenges. Many of its AI-focused peers are enjoying strong investor confidence and significant stock surges directly linked to the AI boom.
*   **Risks:** Apple faces unique risks related to regulatory pressures (antitrust), trade tariffs, and the challenge of accelerating its AI strategy. Its AI infrastructure peers face risks related to market competition and supply chain dynamics. The quantum computing peers carry the highest risk due to the nascent stage of their technology, unproven commercial viability, and high cash burn.

**Overall Conclusion:**

While Apple remains a robust company with strong fundamentals, its outlook is somewhat tempered by its perceived lag in AI innovation and ongoing regulatory challenges. Many of its peers in the computer hardware, storage, and peripherals sub-industry, particularly those directly contributing to AI infrastructure, are currently experiencing more dynamic and significant positive momentum driven by the AI revolution. Apple needs to demonstrate more tangible and impactful AI advancements to fully capitalize on the current technological trends and potentially outperform its more AI-centric peers.

In [44]:
send_message("""What does the recent news say about Apple and the impact of tariffs? From 2025-09-01 up to today. 
Also locate candlestick data for the same dates. 
Discuss in detail any correlations in patterns between the candlestick and news data. Ignore duplicate news entry.""")

send_message: get function response
send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

Here's a detailed analysis of the recent news about Apple and the impact of tariffs, along with candlestick data for the period of September 1, 2025, to November 24, 2025, and a discussion of their correlations.

**News Regarding Apple and Tariffs (September 1, 2025 - November 24, 2025):**

Several news articles during this period discuss the impact of tariffs on Apple, with varying sentiments:

*   **September 3, 2025:** News broke that a U.S. federal judge ruled that Alphabet (Google) does not have to divest Chrome and can continue paying partners like Apple to feature its search engine. This was seen as positive for Apple, with its stock rising over 3% due to the favorable Google lawsuit outcome and a "potential reduction in tariff concerns."
*   **September 4, 2025:** Another article reiterated the positive impact of the Alphabet antitrust ruling on Apple, stating that Apple's stock rallied 4% as its "lucrative search engine deal with Google is expected to continue in some form." However, another article on the same day mentioned Apple "falling behind in AI, facing potential tariff threats, and showing slower revenue growth compared to competitors," indicating a mixed outlook.
*   **September 10, 2025:** Goldman Sachs warned that "Magnificent Seven" tech companies, including Apple, are diverting significant funds to AI data center construction, which is slowing down their share buyback activities. This suggests a strategic shift in capital allocation, potentially influenced by broader economic factors like tariffs.
*   **September 17, 2025:** An article noted that global notebook shipments are expected to grow in 2025, driven by production capacity expansion in Southeast Asia, with Apple "investing large-scale in Vietnam, supporting regional production expansion." This indicates Apple's strategy to mitigate trade risks and diversify its supply chain, likely in response to tariff concerns.
*   **September 18, 2025:** Apple was mentioned as having pledged "$500 billion investment in AI over four years, demonstrating long-term technological commitment." This significant investment could be a strategic move to enhance its competitive edge amidst various market challenges, including tariffs.
*   **October 6, 2025:** An article mentioned Apple "striking side deals to mitigate tariff impacts, indicating strategic adaptability." This highlights Apple's proactive approach to navigating trade tensions.
*   **October 9, 2025:** News reported that "Trump Shocks Markets: VIX Spikes 25%, S&P 500 Eyes Worst Day Since April" due to "President Trump's renewed tariff threats against China." Apple was specifically mentioned as having "experienced significant stock price decline as part of broader market selloff."
*   **October 10, 2025:** Qualcomm's stock dropped after China launched an antitrust investigation, signaling "increased regulatory scrutiny on U.S. tech firms ahead of a potential summit between Presidents Trump and Xi Jinping." Apple was mentioned in this context as potentially losing business, though no direct negative impact was reported.
*   **October 14, 2025:** An article discussed the "Massive News for Stock Market Investors as the Trade War Between the U.S. and China Escalates," mentioning Apple in the disclosure without specific insights.
*   **October 16, 2025:** MP Materials, a rare-earth metal mining company, secured a "$500 million agreement with Apple," positioning itself to support domestic supply chains for advanced technologies. This is a positive development for Apple in securing critical materials, potentially reducing its vulnerability to rare earth element export restrictions.
*   **October 26, 2025:** A news piece titled "Is Apple Going to Be Hit Hard by President Trump's Tariffs?" highlighted "potential significant challenges from US-China trade tensions, particularly around rare earth element export restrictions that could disrupt iPhone component supply chains by November 1st." The sentiment for Apple was "negative" due to these potential impacts.
*   **October 28, 2025:** An article stated that Apple "successfully avoided arduous tariffs through strategic U.S. investments, relocated iPhone production, achieved exemptions from Chinese and Indian tariffs, shares up 53% from April lows." This indicates Apple's successful mitigation strategies. Another article on the same day mentioned Apple preparing for an investor update focusing on "tariff impacts and AI progress," with early indications of strong iPhone 17 sales.
*   **October 31, 2025:** A report on "Global Rare Earth Magnets" mentioned Apple "invested $500M in MP Materials for recycling facility development, proactively addressing supply chain risks." This reinforces Apple's efforts to secure its supply chain against disruptions like tariffs.
*   **November 2, 2025:** An article on MP Materials reiterated Apple's "$500 million in partnership with MP Materials to secure long-term supply of recycled magnets, indicating strategic interest in rare earth metal supply chain."
*   **November 18, 2025:** "The Stock Market Flashes a Warning as Investors Get Bad News About President Trump's Tariffs" reported that Apple "reported $1.1 billion in tariff-related cost increases, expected to rise to $1.4 billion." This indicates a direct financial impact of tariffs on Apple.
*   **November 19, 2025:** Goldman Sachs initiated coverage on MP Materials with a Buy rating, highlighting its strategic importance and mentioning Apple as a partner in magnet supply and recycling with a "$500 million commitment."

**Candlestick Data for Apple (AAPL) (September 1, 2025 - November 24, 2025):**

Here's a summary of the candlestick data, focusing on significant movements:

*   **Early September (Sept 1-4):** The stock opened around $229.25 on Sept 1st and saw a notable increase, closing at $239.69 on Sept 4th.
*   **Mid-September (Sept 5-12):** There was a dip to $226.79 on Sept 7th, followed by a recovery and a significant jump to $245.50 on Sept 12th, and then to $256.08 on Sept 15th.
*   **Late September - Early October (Sept 15 - Oct 9):** The stock generally trended upwards, reaching a high of $277.32 on Oct 3rd, but then experienced a sharp decline, closing at $245.27 on Oct 9th.
*   **Mid-October (Oct 10-16):** The stock recovered some losses, reaching $262.24 on Oct 16th.
*   **Late October - Early November (Oct 17 - Nov 7):** The stock showed some volatility, with a high of $275.96 on Nov 5th, and then a decline to $267.46 on Nov 7th.
*   **Mid-November (Nov 8 - Nov 24):** The stock generally remained in a range, with some fluctuations. The last recorded close on Nov 24th was $271.49.

**Correlations in Patterns between Candlestick and News Data:**

1.  **Early September Rally (Sept 1-4) and Google Antitrust Ruling:** The positive news on September 3rd and 4th regarding the favorable Google antitrust ruling, which was seen to reduce tariff concerns and secure Apple's lucrative search engine deal, directly correlates with the upward movement in Apple's stock price during the first few days of September. The stock rose from around $229 to nearly $240.

2.  **Mid-September Dip (Sept 7) and AI/Tariff Concerns:** While the overall trend in mid-September was upward, a dip to $226.79 on September 7th could be loosely correlated with the news on September 4th mentioning Apple "falling behind in AI, facing potential tariff threats." Although the news was from a few days prior, market sentiment can sometimes have a delayed reaction or be influenced by a confluence of factors.

3.  **October 9th Sharp Decline and Trump's Tariff Threats:** This is a very strong correlation. The news on October 9th explicitly stated that "President Trump's renewed tariff threats against China triggered a significant market selloff," and Apple "experienced significant stock price decline." The candlestick data clearly shows a sharp drop in Apple's stock on October 9th, closing at $245.27 after being in the $250-$270 range in the preceding days.

4.  **Late October - Early November Recovery and Tariff Mitigation Strategies:** Following the sharp decline on October 9th, Apple's stock showed a recovery in late October and early November. This aligns with news on October 28th stating that Apple "successfully avoided arduous tariffs through strategic U.S. investments, relocated iPhone production, achieved exemptions from Chinese and Indian tariffs." The news about Apple's proactive measures to mitigate tariff impacts likely contributed to renewed investor confidence and a rebound in the stock price. Additionally, the news on October 16th and October 31st about Apple's strategic partnership with MP Materials to secure rare earth magnets also likely played a role in this recovery, as it addressed a key supply chain vulnerability related to trade tensions.

5.  **November 18th Tariff-Related Cost Increase and Stock Performance:** The news on November 18th, reporting that Apple "reported $1.1 billion in tariff-related cost increases, expected to rise to $1.4 billion," indicates a direct negative financial impact. While the candlestick data for November 18th shows a close of $267.44, which isn't a drastic drop, it's important to note that the stock had been in a slightly higher range in the days leading up to this, suggesting that the news might have capped further upward movement or contributed to a slight downward pressure.

**Conclusion:**

There are clear correlations between the news regarding tariffs and Apple's stock performance during the specified period. Positive news related to tariff mitigation or favorable rulings (like the Google antitrust case) tended to coincide with upward movements in Apple's stock. Conversely, renewed tariff threats from the U.S. government directly led to a significant decline in Apple's stock. Apple's strategic investments in diversifying its supply chain and relocating production, as reported in the news, appear to have played a crucial role in mitigating some of the negative impacts of tariffs and contributing to stock recoveries. The direct reporting of tariff-related cost increases also had a discernible, albeit sometimes subtle, impact on the stock's trajectory.

# StockChat: Agents Edition

In [ ]:
# Setup working directory on Kaggle.
if os.getenv("KAGGLE_KERNEL_RUN_TYPE"):
    if not os.path.isdir("sc2/"):
        !git init -b main
        !git remote add origin https://github.com/lol-dungeonmaster/kaggle-agents-2025.git
        !git config core.sparseCheckout true
        !echo "sc2/" >> .git/info/sparse-checkout
        !git pull origin main
        for api_key in ["GOOGLE_API_KEY","POLYGON_API_KEY","FINNHUB_API_KEY"]:
            env_key = UserSecretsClient().get_secret(api_key)
            !echo "$api_key=$env_key" >> sc2/.env # from .venv on local runs

In [ ]:
from sc2.src import log
from sc2.agent import app

async def on_event(e: Event, q: str):
    try:
        response = e.content.parts[0].text
        if response and response != "None":
            log.info(f"USER  > {q}")
            log.info(f"MODEL > {response}\n")
    except Exception as err:
        log.error(f"on_event.exception: {str(err)}")

async def run_queries(app: App, sessions: SessionService, queries: list[str],
                      session_id: str = "default", user_id: str = "default"):
    runner = Runner(app=app, session_service=sessions)
    try:
        session = await sessions.create_session(
            app_name=runner.app_name, user_id=user_id, session_id=session_id
        )
    except:
        session = await sessions.get_session(
            app_name=runner.app_name, user_id=user_id, session_id=session_id
        )
    finally:
        log.info(f"### Agent session: (uid={user_id}) {session_id}")
        for q in queries:
            query = types.Content(role="user", parts=[types.Part(text=q)])
            try:
                async for response in runner.run_async(
                    user_id=user_id, session_id=session.id, new_message=query
                ): await on_event(response, q)
            except Exception as e:
                log.error(f"run_async (q={" ".join(q.split()[:5])}): ({type(e).__name__}) {str(e)}")

In [ ]:
inmemory = InMemorySessionService()

await run_queries(
    app=app,
    sessions=inmemory,
    queries=[
        "What tools do you know how to use?",
        "What functions does fncall_pipeline have knowledge about?",
        "What is a short trade?"])